# Business understanding and set-up

## Background and key question

**Airbnb**

Airbnb is an American online platform offering vacation rentals for travelers - primarily homestays - as an alternative to traditional hotel or hostel stays. Additionally, as of late Airbnb also offers "experiences" at popular tourist destinations. For both listings and experiences, Airbnb does not own or host themselves but instead act as an intermediary broker, earning commissions for each booking. Consequently, as many other groundbreaking innovative services Airbnb has not been without controversies, as it offers owners and renters with a great opportunity to make use of unused periods, but incentivizes the dedicated (ab)use of living space as full-time rentals.

**Key Question**

Taking data from a given specific listing time stamp of Berlin, **can we accurately predict its price** in order to provide future hosts with a solid pricing estimate without requiring an Airbnb account beforehand? (aim: offer a tool for new users to get a first pricing and potential earnings indication without requiring an account)

**Assumptions**

As the data is accessible information only and does not include data such as actual occupancy, several assumptions were necessary and prediction could not possibly be conducted with a precision matching a state of complete information. Nevertheless, the wealth of available features made it possible to experiment extensively and work productively on the basis of these assumptions.

**Notes**

The final key question was derived as an iterative process, adapting the target based on the insights from the analysis. Hence, the analysis focuses on evaluating all feasible features, as the decision to include only those that are known to a new user beforehand was made further down the road.

Currently, predictive modeling is performed as a **regression on Target = PRICE_LOG**.

This notebook previously contained predictive modeling for various distinct target options, namely:

- Modeling: Binary Classification (**PRICE_BINARY**)
- Modeling: Multi-Class Classification (**PRICE_CLASS**)
- Modeling: Regression (**PRICE_LOG**)

**As a consequence, the notebook includes target selection via the Dashboard, which is no longer a requirement for the code to be run. Instead, the target variable must be kept as PRICE_LOG (unless you would like to target another feature, which is, however, not intended)**

## Feature glossary

This glossary only lists the key_features used for the final model prediction and does not cover all 105 original features of the dataset.

| **FEATURE** | **DESCRIPTION** |
| :----- | :----- |
| **TARGET: price_log** | log of price per guests_included for a listing |
| **accommodates_per_bed** | created from "accommodates" and "beds" - how many people the property accommodates per available bed |
| **am_balcony** | created from "amenities" - does the listing have a balcony? |
| **am_breakfast** | created from "amenities" - does the rent include breakfast? |
| **am_child_friendly** | created from "amenities" - is the listing flagged as child friendly? |
| **am_elevator** | created from "amenities" - does the listing have an elevator? |
| **am_essentials** | created from "amenities" - does the host provide essentials (e.g. towels, bed linen, ...)? |
| **am_pets_allowed** | created from "amenities" - are pets allowed at the location? |
| **am_private_entrance** | created from "amenities" - do guests have a private entrance? |
| **am_smoking_allowed** | created from "amenities" - is smoking allowed at the location? |
| **am_tv** | created from "amenities" - is a TV available? |
| **bathrooms_log** | log of number of shared (0.5) or dedicated (1.0) bathrooms available |
| **bedrooms** | number of shared (0.5) or dedicated (1.0) bedrooms available |
| **calc_host_lst_count_sqrt_log** | Log of sqrt of number of listings the host owns in total |
| **cancellation_policy** | "flexible", "moderate", "strict" or "super strict" cancellation policies set by the host |
| **guests_included_calc** | number of guests included in the listing price (not necessarily equal to "accommodates", as host may charge for extra_people) |
| **host_is_superhost** | does the host fulfill the criteria and is flagged as a superhost (e.g. high ratings, response rate, ...)? |
| **instant_bookable** | is the listing instant bookable (e.g. without manual confirmation by the host)? |
| **maximum_nights** | what are the maximum  nights specified by the host? |
| **minimum_nights_sqrt** | log of what are the minimum nights specified by the host? |
| **property_type** | which type of property is the listing (e.g. apartment, house, hotel, ...)? |
| **room_type** | which type of room is listed (e.g. entire home/apt, private room, shared room, ...)? |
| **wk_mth_discount** | created from weekly_price and monthly_price - does the host offer a discount for weekly or monthly stays (in %)? |
| **zipcode** | at which zipcode is the listing located? |

## Dataset glossary

| **DATASET** | **DESCRIPTION** |
| :----- | :----- |
| **data_raw** | Originally imported dataset listings.csv.gz (February 2020) |
| **data** | Naming for main working dataset throughout all notebooks |
| **data_clean** | Export from Notebook 1_Clean, import for Notebooks 2_EDA_Clean and 3_Feature_Engineering |
| **data_engineered** | Export from Notebook 3_Feature_Engineering, import for Notebook 4-EDA_Engineered and 5_Predictive_Modeling |
| **best_model_xyz** | Saves of best models from various algorithms |



## Target feature and metric

**Target feature**  
price_log

**Metric**  
neg_median_absolute_error  
  
The analysis and this notebook previously included multiple additional targets and binary as well as multi-class classification predictive models. For the final web application the regression has been selected, which is why the other redundant predictive models have been removed.

## Libraries and dashboard

In [1]:
# Import libraries

# General / Cleaning / Feature Engineering
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import math
import joblib
from numpy import loadtxt
import os, glob
from datetime import date, datetime, timedelta
from currency_converter import CurrencyConverter
%matplotlib inline

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Feature Selection
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from lightgbm import LGBMClassifier

# Predictive Modeling (Models)
from sklearn.dummy import DummyClassifier, DummyRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_predict, cross_val_score, cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, NuSVC, SVR
from sklearn.linear_model import LinearRegression, LogisticRegression, PassiveAggressiveRegressor, ElasticNet, SGDRegressor, RANSACRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor, VotingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, IsolationForest
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from xgboost import XGBClassifier, XGBRegressor
from scipy.stats import randint
from sklearn.multiclass import OneVsRestClassifier
from catboost import CatBoostRegressor

# Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer, fbeta_score, accuracy_score, confusion_matrix, f1_score, precision_recall_curve, recall_score, precision_score, roc_auc_score
from scipy.sparse import csr_matrix
import scipy.stats as stats

# Neural Networks
#from keras import models, layers, optimizers, regularizers
#from keras.utils.vis_utils import model_to_dot
#from keras.wrappers.scikit_learn import KerasRegressor
from IPython.display import SVG

In [2]:
# Dashboard
dataset_loc = "barcelona"  # "berlin", "paris", "amsterdam", "barcelona"
dataset_date = "2020-03-16"  # berlin: "2020-03-17", "2020-06-13", paris: "2020-03-16", amsterdam: "2020-03-14", "2020-07-09", barcelona: "2020-03-16", 
model_run = "2020-09-05"               # date of dataset/model creation (determines subfolder for saves of datasets/models)
target = 'price_log'             # for regression: 'occupancy_rate', 'price_log' | for classification: 'price_class', 'occupancy_class'
# !Important: Please select features for prediction under "Preprocessing"!
scoring = 'neg_median_absolute_error'  # for regression: 'neg_mean_squared_error', 'r2', 'neg_mean_poisson_deviance', 'neg_median_absolute_error' | for classification: "f1(_micro, _macro, _weighted for multiclass)", "recall", "precision", "accuracy", "roc_auc"
test_size = 0.2
random_state = 42
curr = CurrencyConverter()
curr_conv_date=date(2020, 3, 16)

#occ_thr = 0.3  # Threshold for when a listing is deemed a "permanent rental"
review_rate = 0.5  # Assumed share of bookings that were followed up by a user review (feature engineering of occupancy)

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
pd.options.display.max_seq_items = 300
sns.set(style="white")

As mentioned further above, it is necessary to **explicitly declare target** as the notebook was previously set-up to enable analysis and modeling on varying target features.

## Global functions and variables

In [3]:
# "save_load": Function for saving and loading datasets/models (joblib)
def save_load(data=False, title="unknown", file_format="pkl", function="save", dataset_loc=dataset_loc, dataset_date=dataset_date, model_run=model_run):
    if function=="save":
        if file_format=="pkl":
            joblib.dump(data, f"data/{dataset_loc}_{dataset_date}/{model_run}/{title}.pkl")
        elif file_format=="app":
            joblib.dump(data, f"data/{dataset_loc}_{dataset_date}/{title}.pkl")
        else:
            print("Please enter a valid file_format (default is pkl; 'app' for data used in web app).")
    elif function=="load":
        if file_format=="pkl":
            return joblib.load(f"data/{dataset_loc}_{dataset_date}/{model_run}/{title}.pkl")
        elif file_format=="csv":
            return pd.read_csv(f"data/{dataset_loc}_{dataset_date}/{title}.csv")
        elif file_format=="csv.gz":
            return pd.read_csv(f"data/{dataset_loc}_{dataset_date}/{title}.csv.gz")
        elif file_format=="geojson":
            return pd.read_csv(f"data/{dataset_loc}_{dataset_date}/{title}.geojson")
        else:
            print("Please enter a valid file_format (default is pkl).")

In [4]:
# "model_eval": Function for final evaluation of "best model"
def model_eval(y, y_pred, model="reg"):
    """
    Please always specify the type of model:
    Regression: model="reg"
    Binary Classification: model="bclf"
    Multiclass Classification: model="clf"
    """
    if model=="reg":
        print("MSE: {:.2f}".format(mean_squared_error(y, y_pred)))
        print("RMSE: {:.2f}".format(
        mean_squared_error(y, y_pred, squared=False)))
        print("MAE: {:.2f}".format(mean_absolute_error(y, y_pred)))
        print("R2: {:.2f}".format(r2_score(y, y_pred)))
        print("MAPE: {:.2f}".format(mean_absolute_percentage_error(y, y_pred)))
        print("MAPE median: {:.2f}".format(median_absolute_percentage_error(y, y_pred)))

    elif model=="bclf":
        print("Accuracy: {:.2f}".format(accuracy_score(y, y_pred)))
        print("Recall: {:.2f}".format(recall_score(y, y_pred)))
        print("Precision: {:.2f}".format(precision_score(y, y_pred)))
        print("F1 Score: {:.2f}".format(f1_score(y, y_pred)))
        print("ROC/AUC: {:.2f}".format(roc_auc_score(y, y_pred)))
        print("Confusion Matrix: \n" + str(confusion_matrix(y, y_pred)))

    elif model=="clf":
        print("Accuracy: {:.2f}".format(accuracy_score(y, y_pred)))
        print("Recall: {:.2f}".format(recall_score(y, y_pred, average='weighted')))
        print("Precision: {:.2f}".format(precision_score(y, y_pred, average='weighted')))
        print("F1 Score: {:.2f}".format(f1_score(y, y_pred, average='weighted')))
        print("Confusion Matrix: \n" + str(confusion_matrix(y, y_pred)))
    
    else:
        print("Please revise your parameters (e.g. provide a valid model).")

In [5]:
# "print_target_setting": Function for printing current setting for TARGET and the corresponding features
def print_target_setting():
    target_upper = target.upper()
    print(f"You are currently using \033[1m{target_upper}\033[0m as the target and \033[1m{scoring}\033[0m for scoring to predict prices for \033[1m{dataset_loc}\033[0m on \033[1m{dataset_date}\033[0m\n")
    print(f"You are currently using these features for its prediction:\n\033[1m{pred_features}\033[0m\n")
    if 'price_class' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_class'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")
    if 'price_binary' in pred_features:
        print(f"WARNING: Please remove \033[1m'price_binary'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")
    if "occupancy_class" in pred_features and "occupancy_rate" in pred_features:
        print(f"WARNING: Please remove \033[1m'ocupancy_class'\033[0m or \033[1m'ocupancy_rate'\033[0m from features before proceeding with target \033[1m{target_upper}\033[0m")
    else:
        print("No issues with your selection of pred_features have been detected. Please make sure to manually check for correctness nevertheless.")

In [6]:
# "mean_absolute_percentage_error": Function for mean absolute percentage error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [7]:
# "median_absolute_percentage_error": Function for median absolute percentage error (MAPE median)
def median_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

In [8]:
# "get_feat_importances": Function for retrieving feature importances
def get_feat_importances(model, column_names):
    model=model
    feat_importances = pd.DataFrame(model.feature_importances_,
                 columns=['weight'],
                 index=column_names)
    feat_importances.sort_values('weight', inplace=True, ascending=False)
    return feat_importances

In [9]:
# "clf_learning_curves": Function to evaluate classification model based on learning curves
def clf_learning_curves(model):
# Fit model on training data
    model = model
    eval_set = [(X_train_prep, y_train), (X_test_prep, y_test)]
    model.fit(X_train_prep, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)

    # Make predictions for test data
    y_pred = model.predict(X_test_prep)
    predictions = [round(value) for value in y_pred]

    # Evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
    # Retrieve performance metrics
    results = model.evals_result()
    epochs = len(results['validation_0']['error'])
    x_axis = range(0, epochs)
    
    # Plot log loss
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
    ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
    ax.legend()
    pyplot.ylabel('Log Loss')
    pyplot.title('XGBoost Log Loss')
    pyplot.show()
    
    # Plot classification error
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0']['error'], label='Train')
    ax.plot(x_axis, results['validation_1']['error'], label='Test')
    ax.legend()
    pyplot.ylabel('Classification Error')
    pyplot.title('XGBoost Classification Error')
    pyplot.show()

# Data mining

## Data review

The monthly data is composed of various files that are briefly visualized here (based on dataset_loc and dataset_date):

- listings.csv.gz
- listings.csv
- reviews.csv.gz
- reviews.csv
- calendar.csv.gz
- neighbourhoods.csv
- neighbourhoods.geojson

**listings.csv.gz**

In [10]:
# Display contents of listings.csv.gz as well as its shape
#data_listings_gz_insp = save_load(title="listings", file_format="csv.gz", function="load")
#print(data_listings_gz_insp.shape)
#data_listings_gz_insp.head(3)

**listings.csv**

In [11]:
# Display contents of listings.csv as well as its shape
#data_listings_insp = save_load(title="listings", file_format="csv", function="load")
#print(data_listings_insp.shape)
#data_listings_insp.head(2)

**reviews.csv.gz**

In [12]:
# Display contents of reviews.csv.gz as well as its shape
#data_reviews_gz_insp = save_load(title="reviews", file_format="csv.gz", function="load")
#print(data_reviews_gz_insp.shape)
#data_reviews_gz_insp.head(2)

**reviews.csv**

In [13]:
# Display contents of reviews.csv as well as its shape
#data_reviews_insp = save_load(title="reviews", file_format="csv", function="load")
#print(data_reviews_insp.shape)
#data_reviews_insp.head(2)

**calendar.csv.gz**

In [14]:
# Display contents of calendar.csv.gz as well as its shape
#data_cal_insp = save_load(title="calendar", file_format="csv.gz", function="load")
#print(data_cal_insp.shape)
#data_cal_insp.head(2)

**neighbourhoods.csv**

In [15]:
# Display contents of neighbourhoods.csv as well as its shape
#data_neighb_insp = save_load(title="neighbourhoods", file_format="csv", function="load")
#print(data_neighb_insp.shape)
#data_neighb_insp.head(2)

**neighbourhoods.geojson**

In [16]:
# Display contents of neighbourhoods.geojson as well as its shape
#data_neighb_geojson_insp = save_load(title="neighbourhoods", file_format="geojson", function="load")
#print(data_neighb_geojson_insp.shape)
#data_neighb_geojson_insp.head(2)

## Data import

**Create main dataset (listings on January 10th, i.e. pre-COVID-19)**

In [17]:
# Import dataset as DataFrame (as csv-file)
data_raw = save_load(title="listings", file_format="csv.gz", function="load")

C:\Users\Mauricio\anaconda3\envs\airbnb-env\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [18]:
# Assign data_raw to data (in order to always keep a freshly imported data_raw) and set id as index
data = data_raw.copy()
data.set_index('id', inplace=True)

In [19]:
# Create path for saving datasets/models (if not existing)
if not os.path.exists(f"data/{dataset_loc}_{dataset_date}/{model_run}/"):
    os.mkdir(f"data/{dataset_loc}_{dataset_date}/{model_run}/")

# Data cleaning

## Pre-cleaning

Before cleaning the main dataset, the columns were briefly reviewed and all those that were deemed unfruitful for further analysis eliminated beforehand, in order to focus time and effort on the relevant ones.

In [20]:
# Display shape of "data"
data.shape

(21116, 105)

The dataset includes a whopping **105 features** and slightly over 25.000 listings for Berlin for March 2020.

In [21]:
# Display head(1) of "data"
data.head(1)

,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23197,https://www.airbnb.com/rooms/23197,20200316025159,2020-03-16,Forum DeLuxe 5 mins Walk To CCIB Center & Sea★,I do not accept groups of young people under 2...,Elegant spacious apartment suitable for 6. Amp...,I do not accept groups of young people under 2...,none,Strategically located in the area of Parc del ...,"As a Superhost, I aim to make your stay as com...","Tram T4 Campus Diagonal- Besos, over the ro...",You book the entire apartment for yourselves.,"Please do contact Tony, your check-in manager,...",I do not accept groups of young people under 2...,NaN,NaN,https://a0.muscache.com/im/pictures/738532/806...,NaN,90417,https://www.airbnb.com/users/show/90417,Etain (Marnie),2010-03-09,"Catalonia, Spain","Hi. I'm Etain, though everyone calls me by my...",within an hour,100%,100%,t,https://a0.muscache.com/im/users/90417/profile...,https://a0.muscache.com/im/users/90417/profile...,El Besòs i el Maresme,5.0,5.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Sant Adria de Besos, Barcelona, Spain",Sant Martí,el Besòs i el Maresme,Sant Martí,Sant Adria de Besos,Barcelona,08930,Barcelona,"Sant Adria de Besos, Spain",ES,Spain,41.41291,2.22063,f,Apartment,Entire home/apt,6,2.0,3.0,5.0,Real Bed,"{TV,Internet,Wifi,""Wheelchair accessible"",Kitc...",NaN,$220.00,NaN,NaN,$300.00,$80.00,3,$10.00,3,30,3,7,1125,1125,3.2,1125.0,6 days ago,t,30,60,85,343,2020-03-16,52,17,2011-03-15,2019-12-15,95.0,10.0,10.0,10.0,10.0,9.0,9.0,t,HUTB-005057,NaN,t,f,strict_14_with_grace_period,f,t,2,2,0,0,0.47


On first sight, most features appear relatively informative and well-structured. **Amenities** stores lists as a string, which needs to be reviewed later on.

In [22]:
# Display columns of "data"
#data.columns

In [23]:
# Define columns to keep after pre-cleaning
select_columns = [
    'accommodates', 'amenities', 'availability_365', 'availability_90',
    'bathrooms', 'bed_type', 'bedrooms', 'beds',
    'calculated_host_listings_count', 'cancellation_policy', 'cleaning_fee',
    'description', 'experiences_offered', 'extra_people', 'first_review',
    'guests_included', 'has_availability', 'host_acceptance_rate',
    'host_has_profile_pic', 'host_identity_verified', 'host_is_superhost',
    'host_listings_count', 'host_location', 'host_response_rate',
    'host_response_time', 'house_rules', 'instant_bookable', 'interaction',
    'is_business_travel_ready', 'is_location_exact', 'last_review', 'latitude',
    'listing_url', 'longitude', 'maximum_nights', 'minimum_nights',
    'monthly_price', 'name', 'neighborhood_overview', 'neighbourhood_cleansed',
    'notes', 'number_of_reviews', 'number_of_reviews_ltm', 'price',
    'property_type', 'require_guest_phone_verification',
    'require_guest_profile_picture', 'requires_license',
    'review_scores_accuracy', 'review_scores_checkin',
    'review_scores_cleanliness', 'review_scores_communication',
    'review_scores_location', 'review_scores_rating', 'review_scores_value',
    'reviews_per_month', 'room_type', 'security_deposit', 'space',
    'square_feet', 'summary', 'transit', 'weekly_price', 'zipcode'
]

In [24]:
# Drop innecessary columns and sort dataset
drop_columns = [el for el in data.columns if el not in select_columns]
data.drop(labels=drop_columns, inplace=True, axis=1)
data = data.reindex(sorted(data.columns, reverse=False), axis=1)

As mentioned above, around half of the features are kept for further analysis and the rest is dropped - at least for the time being

## Inspection

In [25]:
# Display shape of "data"
data.shape

(21116, 64)

The feature set is now reduced to **64 features**

In [26]:
# Display head(5) of remaining "data"
data.head(5)

,accommodates,amenities,availability_365,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,cancellation_policy,cleaning_fee,description,experiences_offered,extra_people,first_review,guests_included,has_availability,host_acceptance_rate,host_has_profile_pic,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_response_rate,host_response_time,house_rules,instant_bookable,interaction,is_business_travel_ready,is_location_exact,last_review,latitude,listing_url,longitude,maximum_nights,minimum_nights,monthly_price,name,neighborhood_overview,neighbourhood_cleansed,notes,number_of_reviews,number_of_reviews_ltm,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,security_deposit,space,square_feet,summary,transit,weekly_price,zipcode
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23197,6,"{TV,Internet,Wifi,""Wheelchair accessible"",Kitc...",343,85,2.0,Real Bed,3.0,5.0,2,strict_14_with_grace_period,$80.00,I do not accept groups of young people under 2...,none,$10.00,2011-03-15,3,t,100%,t,t,t,5.0,"Catalonia, Spain",100%,within an hour,I do not accept groups of young people under 2...,t,"Please do contact Tony, your check-in manager,...",f,f,2019-12-15,41.41291,https://www.airbnb.com/rooms/23197,2.22063,30,3,NaN,Forum DeLuxe 5 mins Walk To CCIB Center & Sea★,Strategically located in the area of Parc del ...,el Besòs i el Maresme,"As a Superhost, I aim to make your stay as com...",52,17,$220.00,Apartment,t,f,t,10.0,10.0,10.0,10.0,9.0,95.0,9.0,0.47,Entire home/apt,$300.00,Elegant spacious apartment suitable for 6. Amp...,NaN,I do not accept groups of young people under 2...,"Tram T4 Campus Diagonal- Besos, over the ro...",NaN,08930
25786,2,"{TV,Wifi,""Air conditioning"",Kitchen,""Smoking a...",117,52,1.0,Real Bed,1.0,1.0,1,strict_14_with_grace_period,NaN,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,none,$25.00,2010-08-11,1,t,100%,t,t,t,1.0,"Barcelona, Catalonia, Spain",100%,within an hour,Clean Bathroom after use and quick shower At a...,t,Available for interaction. Nonproblem feel fre...,f,t,2020-03-15,41.40145,https://www.airbnb.com/rooms/25786,2.15645,730,1,NaN,NICE ROOM AVAILABLE IN THE HEART OF GRACIA,Solo decir que a menudo ni salgo del barrio. M...,la Vila de Gràcia,No dudes en perdir una cita para un masaje rel...,274,50,$42.00,Apartment,t,t,t,10.0,10.0,9.0,10.0,10.0,95.0,10.0,2.34,Private room,NaN,Room available for rent.- PEDRO PEREZ. Share...,NaN,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,Metro con las principales estaciones. A pie. E...,NaN,08012
31958,4,"{TV,Wifi,""Air conditioning"",Kitchen,Elevator,H...",278,52,1.0,Real Bed,1.0,1.0,30,strict_14_with_grace_period,$58.00,"This apartment has: 1 double bed, 1 double sof...",none,$0.00,2011-02-19,4,t,100%,t,f,f,30.0,"Barcelona, Cataluña, Spain",91%,within an hour,CHECK-IN Week Days: The check-in and key colle...,t,We will be more than happy to help you with an...,f,t,2020-01-07,41.40950,https://www.airbnb.com/rooms/31958,2.15938,27,1,NaN,"Gaudi Attic 1, Friendly Rentals","Looking for apartments in Gràcia, Barcelona fo...",el Camp d'en Grassot i Gràcia Nova,NaN,187,30,$60.00,Apartment,f,f,t,9.0,8.0,9.0,9.0,9.0,91.0,9.0,1.69,Entire home/apt,$200.00,"This apartment has: 1 double bed, 1 double sof...",NaN,"This apartment has: 1 double bed, 1 double sof...",Ideal to discover the city either on foot or b...,NaN,08024
32471,5,"{TV,Wifi,""Air conditioning"",Kitchen,Elevator,H...",268,56,1.5,Real Bed,3.0,3.0,30,strict_14_with_grace_period,$108.00,"This apartment has: 2 double beds, 1 single be...",none,$0.00,2011-01-07,5,t,100%,t,f,f,30.0,"Barcelona, Cataluña, Spain",91%,within an hour,CHECK-IN Week Days: The check-in and key colle...,t,We will be more than happy to he

In [27]:
# Describe data (summary)
data.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
accommodates,21116.0,3.32,2.25,1.00,2.00,2.00,4.00,2.000000e+01
availability_365,21116.0,188.73,127.74,0.00,73.00,201.00,305.00,3.650000e+02
availability_90,21116.0,49.94,33.25,0.00,15.00,57.00,81.00,9.000000e+01
bathrooms,21110.0,1.33,0.67,0.00,1.00,1.00,1.50,1.600000e+01
bedrooms,21093.0,1.55,1.02,0.00,1.00,1.00,2.00,1.600000e+01
beds,20786.0,2.23,1.97,0.00,1.00,2.00,3.00,4.800000e+01
calculated_host_listings_count,21116.0,14.88,31.12,1.00,1.00,2.00,11.00,1.590000e+02
guests_included,21116.0,1.83,1.86,1.00,1.00,1.00,2.00,1.500000e+02
host_listings_count,21109.0,17.72,51.92,0.00,1.00,3.00,11.00,5.320000e+02
latitude,21116.0,41.39,0.01,41.35,41.38,41.39,41.40,4.146000e+01


Data.describe gives a positive impression of the dataset quality. Certain features have missing values and it is noticeable that not all features which should have numeric values show up, which indicates that there may be some issues with datatypes.

In [28]:
# List datatypes (data.info()) (pre-cleaning)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21116 entries, 23197 to 42914062
Data columns (total 64 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   accommodates                      21116 non-null  int64  
 1   amenities                         21116 non-null  object 
 2   availability_365                  21116 non-null  int64  
 3   availability_90                   21116 non-null  int64  
 4   bathrooms                         21110 non-null  float64
 5   bed_type                          21116 non-null  object 
 6   bedrooms                          21093 non-null  float64
 7   beds                              20786 non-null  float64
 8   calculated_host_listings_count    21116 non-null  int64  
 9   cancellation_policy               21116 non-null  object 
 10  cleaning_fee                      16658 non-null  object 
 11  description                       20551 non-null  object 
 1

As expected, various numerical features are currently stored as objects and need to be transformed (e.g. cleaning_fee, extra_people, first_review, price, ...). Additionally, there are actually quite a few columns with missing values.

In [29]:
# Show maximum/minimum value for each numerical column
#num_features = list(data.columns[data.dtypes!=object])
#data[num_features].max()
#data[num_features].min()

Several rows with unusually high values can be identified and may in some cases be dropped at a certain threshold during data handling. Some particular features include:

| **FEATURE** | **MAX_VALUE** |
| :----- | :----- |
| **calculated_host_listings_count** | 57 |
| **accommodates** | 24 |
| **bedrooms** | 12 |
| **beds** | 24 |
| **minimum_nights** | 1.124 |
| **maximum_nights** | 10.000 |
| **number_of_reviews_ltm** | 590 (potentially misleading; actually had less reviews on Airbnb |

In [30]:
# List unique entries per column
data.nunique()

accommodates                           18
amenities                           18568
availability_365                      366
availability_90                        91
bathrooms                              19
bed_type                                5
bedrooms                               15
beds                                   29
calculated_host_listings_count         60
cancellation_policy                     6
cleaning_fee                          148
description                         19344
experiences_offered                     1
extra_people                           80
first_review                         2589
guests_included                        16
has_availability                        1
host_acceptance_rate                   95
host_has_profile_pic                    2
host_identity_verified                  2
host_is_superhost                       2
host_listings_count                    88
host_location                         683
host_response_rate                

Three main insights from unique values:

- Some columns have only 1 value and can be dropped
- Some other columns have 2 values and appear to be true/false (i.e. can be recoded as 1/0)
- Certain columns have a high number of unique values, which can probably be clustered into a few relevant ones (e.g. cancellation_policy, property_type)

In [31]:
# List missing values (pre-cleaning)


def count_missing(data):
    null_cols = data.columns[data.isnull().any(axis=0)]
    X_null = data[null_cols].isnull().sum()
    X_null = X_null.sort_values(ascending=False)
    print(X_null)


count_missing(data)

square_feet                    20614
weekly_price                   19961
monthly_price                  19813
notes                          11503
interaction                     9291
transit                         8488
neighborhood_overview           7265
house_rules                     7257
security_deposit                5995
space                           5819
review_scores_checkin           5419
review_scores_location          5418
review_scores_value             5417
review_scores_accuracy          5414
review_scores_cleanliness       5413
review_scores_communication     5410
review_scores_rating            5406
last_review                     5171
reviews_per_month               5171
first_review                    5171
cleaning_fee                    4458
host_response_time              3434
host_response_rate              3434
host_acceptance_rate            2374
summary                         1128
description                      565
zipcode                          549
b

Various features have a lot of missing values. In particular, there is an observable cut where many features have more than 4.500 missing values and the rest has less than 1.000. The former - except for review_scores - shall be removed, the latter imputed.

**Conclusions (selection)**

- **host_response_rate** and **host_response_time** are unfortunately not available for half of the dataset and consequently the columns are removed
- **review_scores** are difficult to replace if they do not exist, but at 0 they will distort the modeling. Hence, missing values are set to mean of the column
- listings without **name** and the few rows without enhanced **host information** (e.g. superhost), **bedrooms** or **bathrooms** are removed and not substantial in number
- missing values for **summary** and **description** are replaced with "" and kept in order to calculate length during feature engineering
- several features with missing values will be directly converted to 1/0 for simplification (**house_rules, security_deposit, space, cleaning_fee, monthly_price, weekly_price**)

## Define data cleaning functions

**Handle missing/incorrect values**

In [32]:
# Define function for filling missing/incorrect values

def cln_fill_missing_val(data):
   
    # Convert columns with missing values to 1/0
    data.security_deposit.fillna("0", inplace=True)
    data.cleaning_fee.fillna("0", inplace=True)
    data.monthly_price.fillna("0", inplace=True)
    data.weekly_price.fillna("0", inplace=True)

    
    # Fill missing values of "beds" with 0 and then set all with "bed_type" Real Bed to at least 1, those with value "0" to 0.5
    data.beds.fillna(0, inplace=True)
    data.beds = np.where((data.beds == 0) & (data.bed_type == "Real Bed"), 1,
                         data.beds)
    data.beds = np.where((data.beds == 0), 0.5, data.beds)

    
    # Set all with "bathrooms" and "bedrooms" 0 to at least 0.5
    data.bathrooms = np.where(data.bathrooms == 0, 0.5, data.bathrooms)
    data.bedrooms = np.where(data.bedrooms == 0, 0.5, data.bedrooms)

    
    # Fill review_scores with median
    data.review_scores_rating.fillna(data.review_scores_rating.median(),
                                     inplace=True)
    data.review_scores_value.fillna(data.review_scores_value.median(),
                                    inplace=True)
    data.review_scores_checkin.fillna(data.review_scores_checkin.median(),
                                      inplace=True)
    data.review_scores_location.fillna(data.review_scores_location.median(),
                                       inplace=True)
    data.review_scores_communication.fillna(
        data.review_scores_communication.median(), inplace=True)
    data.review_scores_accuracy.fillna(data.review_scores_accuracy.median(),
                                       inplace=True)
    data.review_scores_cleanliness.fillna(
        data.review_scores_cleanliness.median(), inplace=True)

    
    # Fill host_response/acceptance columns with median/"unknown"
    data.host_response_time.fillna("unknown", inplace=True)
    data.host_acceptance_rate.fillna(data.review_scores_rating.median(),
                                     inplace=True)
    data.host_response_rate.fillna(data.review_scores_rating.median(),
                                   inplace=True)

    
    # Fill missing text values with ""
    data.description.fillna("", inplace=True)
    data.interaction.fillna("", inplace=True)
    data.house_rules.fillna("", inplace=True)
    data.neighborhood_overview.fillna("", inplace=True)
    data.notes.fillna("", inplace=True)
    data.space.fillna("", inplace=True)
    data.summary.fillna("", inplace=True)
    data.transit.fillna("", inplace=True)
    
    return data

**Handle wrong/varying datatypes**

In [33]:
# Define function for changing datatypes

def cln_chg_datatypes(data):
    
    # Convert numeric objects to float
    data.cleaning_fee = [
        float(i.strip("$").replace(",", "")) for i in data.cleaning_fee
    ]
    data.extra_people = [
        float(i.strip("$").replace(",", "")) for i in data.extra_people
    ]
    data.host_acceptance_rate = [
        float(str(i).strip("%")) for i in data.host_acceptance_rate
    ]
    data.host_response_rate = [
        float(str(i).strip("%")) for i in data.host_response_rate
    ]
    data.monthly_price = [
        float(i.strip("$").replace(",", "")) for i in data.monthly_price
    ]
    data.price = [float(i.strip("$").replace(",", "")) for i in data.price]
    data.security_deposit = [
        float(i.strip("$").replace(",", "")) for i in data.security_deposit
    ]
    data.weekly_price = [
        float(i.strip("$").replace(",", "")) for i in data.weekly_price
    ]

    
    # Convert varying zipcode datatypes to string
    if dataset_loc == "amsterdam":
        data.zipcode = ["zip_" + str(i)[:4] for i in data.zipcode]
    elif dataset_loc == "barcelona":
        data.zipcode = ["zip_0" + str(i)[:4] for i in data.zipcode]
    else:
        data.zipcode = ["zip_" + str(i)[:5] for i in data.zipcode]   

    
    # Convert date objects to datetime
    data.last_review = np.where(data.last_review > dataset_date,
                                        dataset_date,
                                        data.last_review)
    data = data[data.first_review<=dataset_date]     # Remove listings that had their first review after the official scraping date
    data.first_review = data.first_review.astype('datetime64[D]')
    data.last_review = data.last_review.astype('datetime64[D]')

    
    return data

**Add select amenities as column to data**

In [34]:
# Define function for selecting amenities

def cln_sel_amenities(data):
  
    # Create temporary list with all amenities per listing
    amenities_temp = [
        data.amenities[i].strip("{").strip("}").split(',') for i in data.index
    ]

    
    # Add all amenities to single list in order to count occurrences
    amenities = []
    for lst in amenities_temp:
        for item in lst:
            amenities.append(item)
    amenities = pd.Series(amenities)

    
    # Display count of individual amenities
    #print(amenities.value_counts())

    
    # Add select amenities as distinct columns to data
    data.loc[data.amenities.str.contains('Balcony|Patio'), 'am_balcony'] = 1
    data.am_balcony.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains(
        'Beach view|Beachfront|Lake access|Mountain view|Ski-in/Ski-out|Waterfront'
    ), 'am_nature_and_views'] = 1
    data.am_nature_and_views.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Breakfast'), 'am_breakfast'] = 1
    data.am_breakfast.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('TV'), 'am_tv'] = 1
    data.am_tv.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Coffee maker|Espresso machine'),
             'am_coffee_machine'] = 1
    data.am_coffee_machine.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Cooking basics'),
             'am_cooking_basics'] = 1
    data.am_cooking_basics.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Dishwasher|Dryer|Washer'),
             'am_white_goods'] = 1
    data.am_white_goods.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Elevator'), 'am_elevator'] = 1
    data.am_elevator.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Essentials'), 'am_essentials'] = 1
    data.am_essentials.fillna(0, inplace=True)

    data.loc[
        data.amenities.str.contains('Family/kid friendly|Children|children'),
        'am_child_friendly'] = 1
    data.am_child_friendly.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('parking'), 'am_parking'] = 1
    data.am_parking.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Pets|pet|Cat(s)|Dog(s)'),
             'am_pets_allowed'] = 1
    data.am_pets_allowed.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Private entrance'),
             'am_private_entrance'] = 1
    data.am_private_entrance.fillna(0, inplace=True)

    data.loc[data.amenities.str.contains('Smoking allowed'),
             'am_smoking_allowed'] = 1
    data.am_smoking_allowed.fillna(0, inplace=True)
    
    return data

Out of the full list of amenities, not all will have a significant impact on the price. For the purpose of this analysis, an initial selection has been made and then enhanced by some great [previous work](https://github.com/L-Lewis/Airbnb-neural-network-price-prediction/blob/master/Airbnb-price-prediction.ipynb) on selecting relevant amenities. Additionally, most amenities with a split of more than 90/10 between 1/0 have been **removed (strikethrough in the list)** - except for some that were deemed substantial (24-hour check-in, breakfast, essentials, nature and views)

| **NEW COLUMN** | **PREVIOUS AMENITY/IES** |
| :----- | :----- |
| <s>**am_check_in_24h**</s> | <s>24-hour check-in</s> |
| **<s>am_air_con</s>** | <s>Air conditioning/central air conditioning</s> |
| **am_balcony** | Balcony/patio or balcony |
| **am_nature_and_views** | Beach view/beachfront/lake access/mountain view/ski-in ski-out/waterfront (i.e. great location/views) |
| **am_breakfast** | Breakfast |
| **am_tv** | Cable TV/TV |
| **am_coffee_machine** | Coffee maker/espresso machine |
| **am_cooking_basics** | Cooking basics |
| **am_white_goods** | Dishwasher/Dryer/Washer/Washer and dryer |
| **am_elevator** | Elevator |
| <s>**am_gym**</s> | <s>Exercise equipment/gym/private gym/shared gym</s> |
| **am_essentials** | Essentials |
| **am_child_friendly** | Family/kid friendly, or anything containing 'children' |
| **am_parking** | Free parking on premises/free street parking/outdoor parking/paid parking off premises/paid parking on premises |
| <s>**am_outdoor_space**</s> | <s>Garden or backyard/outdoor seating/sun loungers/terrace</s> |
| <s>**am_wellness**</s> | <s>Hot tub/jetted tub/private hot tub/sauna/shared hot tub/pool/private pool/shared pool</s> |
| <s>**am_internet**</s> | <s>Internet/pocket wifi/wifi</s> |
| **am_pets_allowed** | Pets allowed/cat(s)/dog(s)/pets live on this property/other pet(s) |
| **am_private_entrance** | Private entrance |
| <s>**am_secure**</s> | <s>Safe/security system</s> |
| <s>**am_self_check_in**</s> | <s>Self check-in</s> |
| **am_smoking_allowed** | Smoking allowed |

**Drop irrelevant rows**

Many of the decisions below are based on assumptions and judgement derived from the analysis, EDA and intuition.

In [35]:
# Define function for dropping irrelevant rows

def cln_drop_rows(data):
    
    # Drop missing rows of features with few missing values
    data.dropna(subset=[
        "name", "host_is_superhost", "bedrooms", "bathrooms",
        "neighbourhood_cleansed", "zipcode"
    ],
                inplace=True)

    
    # Drop missing rows of zipcode
    data = data[data.zipcode != "zip_nan"]
    data = data[data.zipcode != "zip_0nan"]

    
    # Remove "poor" listings (value above/below a certain threshold)
    data = data[data.price < 500]
    data = data[data.price >= 10]
    data = data[data.minimum_nights <= 100]

    
    # Remove listings where "accommodates" is lower than "guests_included"
    data = data[data.accommodates - data.guests_included >= 0]

    
    # Remove listings where "accommodates" > 10 (outliers)
    data = data[data.accommodates <= 10]

    
    # Remove listings where "accommodates" - "beds" < 0
    data = data[data.accommodates - data.beds >= 0]

    
    # Remove listings where "bedrooms" - "beds" > 2
    data = data[data.bedrooms - data.beds <= 2]

    
    # Remove listings where "beds" - "bedrooms" > 10
    data = data[data.beds - data.bedrooms <= 10]

    
    # Remove listings where "monthly_price" is more than 30x "price"
    data = data[data.monthly_price / data.price <= 30]

    
    # Remove listings where "weekly_price" is more than 7x "price"
    data = data[data.weekly_price / data.price <= 7]

    
    # Remove "inactive" or "new" listings with no reviews in last twelve months
    data = data[data.number_of_reviews_ltm != 0]

    
    # Remove listings with no "availability_365" and no reviews in last three months
    data = data[(data.availability_365 != 0) |
                (data.last_review > ((datetime.strptime(dataset_date, "%Y-%m-%d"))-timedelta(3 * 30)).strftime("%Y-%m-%d"))] # Calculate 90 days backwards from dataset_date
    
    return data

In [ ]:
# FUTURE WORK: Do row limitation based on data.describe() and remove through one single function (adapt code below)
#def preprocess(trips_in):
#    trips = trips_in.copy(deep=True)
#    trips.fare_amount = trips.fare_amount + trips.tolls_amount
#    del trips["tolls_amount"]
#    del trips["total_amount"]
#    del trips["trip_distance"] # we won't know this in advance!

#    qc = np.all([
#        trips["pickup_longitude"] > -78,
#        trips["pickup_longitude"] < -70,
#        trips["dropoff_longitude"] > -78,
#        trips["dropoff_longitude"] < -70,
#        trips["pickup_latitude"] > 37,
#        trips["pickup_latitude"] < 45,
#        trips["dropoff_latitude"] > 37,
#        trips["dropoff_latitude"] < 45,
#        trips["passenger_count"] > 0
#    ], axis=0)

#    return trips[qc]

#tripsqc = preprocess(trips)
#tripsqc.describe()

**Drop irrelevant columns**

Remove low-frequency classes from categorical columns: Neighbourhoods_cleansed and zipcodes both have a high volume of unique values, many with under 10 occurrences. Hence, all with <0.25% share are bundled as "other".

In [36]:
# Define function for dropping irrelevant columns

def cln_drop_cols(data):    
    
    # Change neighbourhoods_cleansed that make up <0.25% of data to "other"
    data = data.apply(lambda x: x.mask(
        x.map(x.value_counts()) < (0.0025 * len(data)), 'nb_other')
                      if x.name == 'neighbourhood_cleansed' else x)
    
    
    # Change zipcodes that make up <0.25% of data to "other"
    data = data.apply(lambda x: x.mask(
        x.map(x.value_counts()) < (0.0025 * len(data)), 'zip_other')
                      if x.name == 'zipcode' else x)
    
    
    # Drop irrelevant columns
    data.drop(
        [
            "bed_type",
            "experiences_offered",
            "has_availability",
    #        "host_acceptance_rate",
            "host_location",
    #        "host_response_rate",
    #        "host_response_time",  
    #        "number_of_reviews", 
    #        "number_of_reviews_ltm",
            "requires_license",
            "is_business_travel_ready",
            "host_has_profile_pic",
            "host_listings_count",
            "require_guest_profile_picture",
            "require_guest_phone_verification",
            "reviews_per_month",
            "square_feet"
        ],
        inplace=True,
        axis=1)
    
    return data

Explanation for selection of dropped columns:

| **FEATURE(S)** | **NOTES** |
| :----- | :----- | 
| **bed_type** | over 97% of values were "Real Bed", hence little added value |
| **experiences_offered** | all values are "none" |
| **has_availability** | all values are "t" |
| **requires_license, host_has_profile_pic** | almost all values are "t" |
| **is_business_travel_ready** | all values are "f" |
| **require_guest_xxx** | almost all values are "f" |
| **host_listings_count** | calculated_host_listings_count appears to be a sanitized version (ranges from 1 to 55) of host_listings_count (has values 0 and highest is 1397) |
 **other host_xyz** | too many missing values |
| **reviews_per_month** | number_of_reviews_ltm kept as better measure |
| **square_feet** | too many missing values |
| <s>**property_type**</s> | <s>90% of values are "apartment", too many unique values to sensibly classify</s> kept instead |

## Apply data cleaning functions

In [37]:
# Bundle data cleaning steps as function "data_cleaning"
def data_cleaning(data):
    data = cln_fill_missing_val(data)
    data = cln_chg_datatypes(data)
    data = cln_sel_amenities(data)
    data = cln_drop_rows(data)
    data = cln_drop_cols(data)
    
    return data

In [38]:
# Apply data cleaning to dataset
data = data_cleaning(data)

C:\Users\Mauricio\anaconda3\envs\airbnb-env\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\Mauricio\anaconda3\envs\airbnb-env\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Mauricio\anaconda3\envs\airbnb-env\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

## Final check and export

In [39]:
# Sort columns in dataset
data = data.reindex(sorted(data.columns, reverse=False), axis=1)

In [40]:
# List datatypes (data.info()) (post-cleaning)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11842 entries, 23197 to 42833940
Data columns (total 66 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   accommodates                    11842 non-null  int64         
 1   am_balcony                      11842 non-null  float64       
 2   am_breakfast                    11842 non-null  float64       
 3   am_child_friendly               11842 non-null  float64       
 4   am_coffee_machine               11842 non-null  float64       
 5   am_cooking_basics               11842 non-null  float64       
 6   am_elevator                     11842 non-null  float64       
 7   am_essentials                   11842 non-null  float64       
 8   am_nature_and_views             11842 non-null  float64       
 9   am_parking                      11842 non-null  float64       
 10  am_pets_allowed                 11842 non-null  float64       


Many columns are only kept for EDA or feature engineering and will be dropped afterwards. The remaining columns are prepared and ready for further processing.

In [41]:
# List missing values (post-cleaning)

def count_missing(data):
    null_cols = data.columns[data.isnull().any(axis=0)]
    X_null = data[null_cols].isnull().sum()
    X_null = X_null.sort_values(ascending=False)
    print(X_null)

count_missing(data)
#data.isnull().sum()

Series([], dtype: float64)


As we can see, we got rid of all the missing values

In [42]:
# Display cleaned dataset
print(data.shape)
data.head(3)

(11842, 66)


,accommodates,am_balcony,am_breakfast,am_child_friendly,am_coffee_machine,am_cooking_basics,am_elevator,am_essentials,am_nature_and_views,am_parking,am_pets_allowed,am_private_entrance,am_smoking_allowed,am_tv,am_white_goods,amenities,availability_365,availability_90,bathrooms,bedrooms,beds,calculated_host_listings_count,cancellation_policy,cleaning_fee,description,extra_people,first_review,guests_included,host_acceptance_rate,host_identity_verified,host_is_superhost,host_response_rate,host_response_time,house_rules,instant_bookable,interaction,is_location_exact,last_review,latitude,listing_url,longitude,maximum_nights,minimum_nights,monthly_price,name,neighborhood_overview,neighbourhood_cleansed,notes,number_of_reviews,number_of_reviews_ltm,price,property_type,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,security_deposit,space,summary,transit,weekly_price,zipcode
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23197,6,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,"{TV,Internet,Wifi,""Wheelchair accessible"",Kitc...",343,85,2.0,3.0,5.0,2,strict_14_with_grace_period,80.0,I do not accept groups of young people under 2...,10.0,2011-03-15,3,100.0,t,t,100.0,within an hour,I do not accept groups of young people under 2...,t,"Please do contact Tony, your check-in manager,...",f,2019-12-15,41.41291,https://www.airbnb.com/rooms/23197,2.22063,30,3,0.0,Forum DeLuxe 5 mins Walk To CCIB Center & Sea★,Strategically located in the area of Parc del ...,el Besòs i el Maresme,"As a Superhost, I aim to make your stay as com...",52,17,220.0,Apartment,10.0,10.0,10.0,10.0,9.0,95.0,9.0,Entire home/apt,300.0,Elegant spacious apartment suitable for 6. Amp...,I do not accept groups of young people under 2...,"Tram T4 Campus Diagonal- Besos, over the ro...",0.0,zip_other
25786,2,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Smoking a...",117,52,1.0,1.0,1.0,1,strict_14_with_grace_period,0.0,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,25.0,2010-08-11,1,100.0,t,t,100.0,within an hour,Clean Bathroom after use and quick shower At a...,t,Available for interaction. Nonproblem feel fre...,t,2020-03-15,41.40145,https://www.airbnb.com/rooms/25786,2.15645,730,1,0.0,NICE ROOM AVAILABLE IN THE HEART OF GRACIA,Solo decir que a menudo ni salgo del barrio. M...,la Vila de Gràcia,No dudes en perdir una cita para un masaje rel...,274,50,42.0,Apartment,10.0,10.0,9.0,10.0,10.0,95.0,10.0,Private room,0.0,Room available for rent.- PEDRO PEREZ. Share...,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,Metro con las principales estaciones. A pie. E...,0.0,zip_00801
31958,4,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,Elevator,H...",278,52,1.0,1.0,1.0,30,strict_14_with_grace_period,58.0,"This apartment has: 1 double bed, 1 double sof...",0.0,2011-02-19,4,100.0,f,f,91.0,within an hour,CHECK-IN Week Days: The check-in and key colle...,t,We will be more than happy to help you with an...,t,2020-01-07,41.40950,https://www.airbnb.com/rooms/31958,2.15938,27,1,0.0,"Gaudi Attic 1, Friendly Rentals","Looking for apartments in Gràcia, Barcelona fo...",el Camp d'en Grassot i Gràcia Nova,,187,30,60.0,Apartment,9.0,8.0,9.0,9.0,9.0,91.0,9.0,Entire home/apt,200.0,"This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...",Ideal to discover the city either on foot or b...,0.0,zip_00802


The number of listings has been reduced substantially by removing rows deemed to be irrelevant or distorting.

**Export data_clean**

In [43]:
# Export dataset for further use in 2_EDA_Clean and 3_Feature_Engineering
data_clean = data.copy()
save_load(data_clean, title="data_clean", function="save")

# Feature engineering

In [44]:
# Import data_clean
data = save_load(title="data_clean", function="load")

In [45]:
# Import reviews.csv, convert date to datetime and remove all that are more recent than dataset_date
data_rev = save_load(title="reviews", file_format="csv", function="load")
data_rev.date = data_rev.date.astype('datetime64[D]')
data_rev = data_rev[data_rev.date<=dataset_date]
#print(data_rev.shape)
#data_rev.head(3)

## Define feature engineering functions

**Change column content**

- Reduce cancellation_policy and property_type classes
- Replace "0" values in monthly_price and weekly_price
- Recalculate guests_included (many listings specify guests_included=1 while accommodates is higher and no extra fee is charged)

In [46]:
# Define function for adapting existing features

def feat_adapt(data):

    # Reduce cancellation_policy to 4 classes
    data.cancellation_policy.replace(
        ["strict_14_with_grace_period", "super_strict_60", "super_strict_30"],
        ["strict", "super_strict", "super_strict"],
        inplace=True)

    
    # Reduce property_type to 6 classes, as per Airbnb classification (see listing creation in pdf)
    data.property_type.replace(["Condominium", "Loft", "Vacation home"],
                               "Apartment",
                               inplace=True)
    data.property_type.replace(
        ["Aparthotel", "Hostel", "Hotel", "Resort", "Serviced apartment"],
        "Boutique hotel",
        inplace=True)
    data.property_type.replace([
        "Casa particular (Cuba)", "Farm stay", "Nature lodge",
        "Pension (South Korea)"
    ],
                               "Bed and breakfast",
                               inplace=True)
    data.property_type.replace([
        "Bungalow", "Cabin", "Chalet", "Cottage", "Dome house", "Earth house",
        "Houseboat", "Hut", "Lighthouse", "Tiny house", "Townhouse", "Villa"
    ],
                               "House",
                               inplace=True)
    data.property_type.replace(["Guesthouse", "Guest suite"],
                               "Secondary unit",
                               inplace=True)
    data.property_type.replace([
        "Barn", "Boat", "Bus", "Camper/RV", "Campsite", "Castle", "Cave",
        "Igloo", "Island", "Plane", "Tent", "Tipi", "Train", "Treehouse",
        "Windmill", "Yurt"
    ],
                               "Unique space",
                               inplace=True)

    
    # Drop all listings that are not in the above 6 classes
    data = data[data.property_type.isin([
        "Apartment", "Boutique hotel", "Bed and breakfast", "House",
        "Secondary unit", "Unique space"
    ])]

    
    # Fill all columns with "0" of "monthly_price" and "weekly_price" with 7x/30x "price"
    data["monthly_price"] = np.where(data.monthly_price == 0, data.price * 30,
                                     data.monthly_price)
    data["weekly_price"] = np.where(data.weekly_price == 0, data.price * 7,
                                    data.weekly_price)

    
    # Re-calculate "guests_included_calc" to be identical to "accommodates" where "extra_people"==0
    data["guests_included_calc"] = np.where(data.extra_people == 0,
                                            data.accommodates,
                                            data.guests_included)
    
    # Convert price from USD to EUR
    data.price = [round(curr.convert(el, 'USD', 'EUR', date=curr_conv_date), 2) for el in data.price]
    
    return data

**Convert binary features to 1/0**

In [47]:
# Define function for engineering binary features

def feat_bin(data):
    
    # Convert t/f to 1/0 for various features
    data.host_is_superhost.replace(["t", "f"], [1, 0], inplace=True)
    data.host_identity_verified.replace(["t", "f"], [1, 0], inplace=True)
    data.is_location_exact.replace(["t", "f"], [1, 0], inplace=True)
    data.instant_bookable.replace(["t", "f"], [1, 0], inplace=True)
    
    
    # Change availability_365 to 1/0
    data.availability_365 = np.where(data.availability_365 != 0, 1, 0)
    
    
    # Create 1/0 for text descriptions
    data["description_exist"] = np.where(data.description != "", 1, 0)
    data["house_rules_exist"] = np.where(data.house_rules != "", 1, 0)
    data["interaction_exist"] = np.where(data.interaction != "", 1, 0)
    data["neighborhood_overview_exist"] = np.where(
        data.neighborhood_overview != "", 1, 0)
    data["notes_exist"] = np.where(data.notes != "", 1, 0)
    data["space_exist"] = np.where(data.space != "", 1, 0)
    data["summary_exist"] = np.where(data.summary != "", 1, 0)
    data["transit_exist"] = np.where(data.transit != "", 1, 0)
     
    return data

**Create numerical features**

In [48]:
# Define function for engineering numerical features

def feat_num(data):

    # Retrieve "listing_no" from "listing_url"
    data["listing_no"] = [int(el.split("/")[-1]) for el in data.listing_url]

    
    # Calculate "price_calc" for one person from "price", "guests_included", "extra_people" and remove listings where "price_calc" ends up being <= 5
    data["price_calc"] = data.price - 0.5 * data.extra_people * (
        data.guests_included - 1)
    data = data[data.price_calc > 5]

    
    # Calculate "price_extra_people" (price) for additional persons from "price", "guests_included", "extra_people" and "accommodates"
    data["price_extra_people"] = (
        data.extra_people * (data.accommodates - data.guests_included) +
        (0.5 * data.extra_people *
         (data.guests_included - 1))) / (data.accommodates - 1)
    data.price_extra_people.fillna(0, inplace=True)

    
    # Calculate occurrence of "price_extra_fees" from "security_deposit" and "cleaning_fee"
    data["price_extra_fees"] = 0 + data.security_deposit + data.cleaning_fee

    
    # Calculate "descr_detail" as measure for how well the listing is described
    data[
        "descr_detail"] = 0 + data.description_exist + data.house_rules_exist + data.interaction_exist + data.neighborhood_overview_exist + data.notes_exist + data.space_exist + data.summary_exist + data.transit_exist
    
    
    # Calculate "accommodates_per_bed" as feature to de-correlate "accommodates", "beds" and "bedrooms"
    data["accommodates_per_bed"] = data.accommodates / data.beds
    
    
    # Calculate "wk_mth_discount" from "monthly_price" and "weekly_price" with "price"
    data["wk_mth_discount"] = ((data.price * 30 - data.monthly_price) /
                               (data.price * 30) +
                               (data.price * 7 - data.weekly_price) /
                               (data.price * 7)) / 2

    
    # Calculate days since "first_review_days"
    data["first_review_days"] = (datetime.strptime(dataset_date,
                                                   '%Y-%m-%d')) - data.first_review
    data.first_review_days = [i.days for i in data.first_review_days]
    
    
    # Calculate days since "last_review_days"
    data["last_review_days"] = (datetime.strptime(dataset_date,
                                                  '%Y-%m-%d')) - data.last_review
    data.last_review_days = [i.days for i in data.last_review_days]
    
    
    # Calculate "review_scores_calc" as proxy considering number of reviews and penalizing new/inactive listings
    new_bias = [math.sqrt(el/50) for el in data.last_review_days]
    data["review_scores_calc"] = data.review_scores_rating - new_bias
    new_bias = []
    for reviews in data.number_of_reviews_ltm:
        if reviews < 10:
            new_bias.append(-3 + math.sqrt(reviews))
        else:
            new_bias.append(0)
    data.review_scores_calc = data.review_scores_calc + new_bias
        
    return data

**Create categorical features**

In [49]:
# Define function for engineering categorical features

def feat_cat(data):
    
    # Categorize listings by "state" (basic, moderate, luxurious)
    
    
    # Create "text_len" as word count from text-based (.split().count)
    data["description_len"] = [len(i.split()) for i in data.description]
    data["house_rules_len"] = [len(i.split()) for i in data.house_rules]
    data["interaction_len"] = [len(i.split()) for i in data.interaction]
    data["neighborhood_overview_len"] = [
        len(i.split()) for i in data.neighborhood_overview
    ]
    data["notes_len"] = [len(i.split()) for i in data.notes]
    data["space_len"] = [len(i.split()) for i in data.space]
    data["summary_len"] = [len(i.split()) for i in data.summary]
    data["transit_len"] = [len(i.split()) for i in data.transit]
    data["text_len"] = (
        data.description_len / data.description_len.max() +
        data.house_rules_len / data.house_rules_len.max() +
        data.interaction_len / data.interaction_len.max() +
        data.neighborhood_overview_len / data.neighborhood_overview_len.max() +
        data.notes_len / data.notes_len.max() + data.space_len /
        data.space_len.max() + data.summary_len / data.summary_len.max() +
        data.transit_len / data.transit_len.max()) / 8
    data.text_len = data.text_len / data.text_len.max()
    
    
    # Categorize listings as "review_scores_class" by "review_scores_rating"
    review_scores_class = []
    for score in data.review_scores_rating:
        if score == 0:
            review_scores_class.append(0)
        elif score <= 89:
            review_scores_class.append(1)
        elif score <= 93:
            review_scores_class.append(2)
        elif score <= 96:
            review_scores_class.append(3)
        elif score <= 99:
            review_scores_class.append(4)
        else:
            review_scores_class.append(5)
    data["review_scores_class"] = review_scores_class
    
    
    # Categorize listings as "review_scores_class_new" by "review_scores_calc"
    review_scores_class_new = []
    for score in data.review_scores_calc:
        if score == 0:
            review_scores_class_new.append(0)
        elif score <= 89:
            review_scores_class_new.append(1)
        elif score <= 92.5:
            review_scores_class_new.append(2)
        elif score <= 96:
            review_scores_class_new.append(3)
        elif score <= 98:
            review_scores_class_new.append(4)
        else:
            review_scores_class_new.append(5)
    data["review_scores_class_new"] = review_scores_class_new
    
    
    # Categorize listings as "price_class" by "price_calc"
    price_class = []
    for price in data.price_calc:
        if price <= 20:
            price_class.append(1)
        elif price <= 30:
            price_class.append(2)
        elif price <= 40:
            price_class.append(3)
        elif price <= 50:
            price_class.append(4)
        elif price <= 60:
            price_class.append(5)
        elif price <= 70:
            price_class.append(6)
        elif price <= 80:
            price_class.append(7)
        elif price <= 90:
            price_class.append(8)
        elif price <= 100:
            price_class.append(9)
        elif price <= 150:
            price_class.append(10)
        else:
            price_class.append(11)
    data["price_class"] = price_class
        
    return data

**Convert text columns into meaningful information**

In [50]:
# Define function for engineering text features

def feat_text(data):

    # FUTURE WORK: data.description.sample(5)
    pass
    
    return data

**Create log/sqrt from existing features**

Now we will replace certain features, which have relatively high skew (see 2_Clean), with their log or sqrt

In [51]:
# Define function for creating log/sqrt from skewed features

def feat_log_sqrt(data):
    
    # Create log "bathrooms_log" for numerical feature "bathrooms"
    data["bathrooms_log"] = [math.log(el) for el in data["bathrooms"]]
    
    
    # Create sqrt and log "calc_host_lst_count_sqrt_log" for numerical feature "calculated_host_listings_count"
    data["calc_host_lst_count_sqrt_log"] = [
        math.log(math.sqrt(el)) for el in data["calculated_host_listings_count"]
    ]
    
    
    # Create sqrt "first_review_days_sqrt" for numerical feature "first_review_days"
    data["first_review_days_sqrt"] = [
        math.sqrt(el) for el in data.first_review_days
    ]
    
    
    # Create sqrt "last_review_days_sqrt" for numerical feature "last_review_days"
    data["last_review_days_sqrt"] = [math.sqrt(el) for el in data.last_review_days]
    
    
    # Create sqrt "minimum_nights_sqrt" for numerical feature "minimum_nights"
    data["minimum_nights_sqrt"] = [math.sqrt(el) for el in data["minimum_nights"]]
    
    
    # Create log "number_of_reviews_ltm_log" for numerical feature "number_of_reviews_ltm"
    data["number_of_reviews_ltm_log"] = [
        math.sqrt(el) for el in data["number_of_reviews_ltm"]
    ]
    
    
    # Create log "price_extra_fees_sqrt" for numerical feature "price_extra_fees"
    data["price_extra_fees_sqrt"] = [
        math.sqrt(el) for el in data["price_extra_fees"]
    ]
    
    
    # Create log "price_log" for numerical feature "price"
    data["price_log"] = [math.log(el) for el in data["price"]]
    
    
    # Create log "price_calc_log" for numerical feature "price_calc"
    data["price_calc_log"] = [math.log(el) for el in data["price_calc"]]
    
    
    # Create sqrt "review_scores_rating_sqrt" for numerical feature "review_scores_rating"
    review_max = data.review_scores_rating.max()
    data["review_scores_rating_sqrt"] = [
        math.sqrt(review_max - el) for el in data.review_scores_rating
    ]
    review_log_max = data.review_scores_rating_sqrt.max()
    data["review_scores_rating_sqrt"] = [(review_log_max - el)
                                         for el in data.review_scores_rating_sqrt]
    #data["review_scores_rating_sqrt"].plot(kind='hist', bins=50, figsize=(12,6), facecolor='grey',edgecolor='black');
    
    
    # Create log "text_len_sqrt" for numerical feature "text_len"
    data["text_len_sqrt"] = [math.sqrt(el) for el in data["text_len"]]
        
    return data

**Calculate occupancy rate**

**Occupancy_rate initially played a major role in the consideration of creating the predictive model. It was, however, deemed too much of an insecure variable to be included in price prediction - or the other way around. It is still kept in this notebook for reference and potentially future work.**

Calculation of **occupancy rate** is inspired by the **San Francisco model**, which is also applied by [Inside AirBnB](http://insideairbnb.com/about.html):

- (**A**) Determine the **average length of stay for Berlin**
- (**B**) Calculate **reviews relevant for considered timeframe**
- (**C**) Determine **active months in timeframe** from price (not relevant if only 1 month)
- (**D**) Estimate **# of bookings in considered timeframe** using (**B**)
- (**E**) **Occupancy rate** = (**D**)x(**A**) / ((**C**)/months x time span)

Read more about the core idea behind the calculations of the model [here](https://sfbos.org/sites/default/files/FileCenter/Documents/52601-BLA.ShortTermRentals.051315.pdf). Assumptions were adapted for the purpose of this analysis, mainly due to the core idea of considering only the two most recent years.

**Notes**:
- **(A)**: For the purpose of this model, around **3 nights** are assumed as average length of stay in Berlin and used as basis for calculation, unless a higher minimum length is specified. Back in 2016, [4.6](https://www.airbnbcitizen.com/wp-content/uploads/2016/04/airbnb-community-berlin-en.pdf) has been reported as the average length of stay. Inside AirBnB uses 3 nights for cities where no current data is available, but uses [6.3 nights](http://insideairbnb.com/berlin/#) for its Berlin visualization
- **(D)**: Estimate **# of bookings in considered timeframe** by dividing (**B**) through an assumed 50% review rate (i.e. one review corresponds to two bookings)

In [52]:
# Define function for calculating occupancy rate based on formula above

def feat_occupancy(data): 

    # (**A**) Determine the **average length of stay for Berlin**
    # Add column to main dataframe for avg length of stay, being either a) 5 nights or b) minimum_nights if higher than 5 or c) avg of min and max if max is 5 or lower
    avg_nights = []
    for idx in data.index:
        if data.maximum_nights[idx] <= 5:
            avg_nights.append(
                (data.maximum_nights[idx] + data.minimum_nights[idx]) / 2)
        elif data.minimum_nights[idx] > 3:
            avg_nights.append(data.minimum_nights[idx])
        else:
            avg_nights.append(3)
    data["avg_nights"] = avg_nights
    
    
    # (**B**) Calculate **reviews in considered timeframe**
    # Keep only reviews within a specified timeframe (see Dashboard)
    data_rev_count = data_rev[(data_rev.date > ((datetime.strptime(dataset_date, "%Y-%m-%d"))-timedelta(3 * 30)).strftime("%Y-%m-%d"))
                              & (data_rev.date < dataset_date)]
    data_rev_count = pd.DataFrame(data_rev_count.listing_id.value_counts()
                                  )  # Count reviews per listing and save as table
    
    # Merge review count to "data"
    data_rev_count.rename(columns={"listing_id": "reviews_3mth"},
                          inplace=True)  # Change column name
    data = pd.merge(data,
                    data_rev_count,
                    how="left",
                    left_index=True,
                    right_index=True)  # Add column to main dataset
    data.reviews_3mth.fillna(0, inplace=True)
    
    
    # (**C**) Determine **active months and relevant months** from price
    # Count the months where listings were online with a price (not relevant if 1 mth)
    data["active_months"] = 1
    relevant_mths = 1
    
    
    # (**D**) Estimate **# of bookings in considered timeframe**
    # Calculate bookings estimate and replace NaN with 0
    data["bookings_est"] = data.reviews_3mth / review_rate
    data.bookings_est.fillna(0, inplace=True)
    
    
    # (**E**) **Occupancy rate** = (**D**)x(**A**) / ((**C**)/months x time span)
    # Calculate occupancy rate
    data["occupancy_rate"] = data.bookings_est * data.avg_nights / (
        data.active_months / relevant_mths * 90)
    
    
    # Modify occupancy rate
    # Cap occupancy at 100%
    occupancy_temp = []
    for rate in data.occupancy_rate:
        if rate < 1:
            occupancy_temp.append(rate)
        else:
            occupancy_temp.append(1)
    data.occupancy_rate = occupancy_temp
    
    # Split occupancy into 2 classes according to threshold (splitting into temporary and permanent rentals)
    occupancy_class = []
    for rate in data.occupancy_rate:
        if rate < 0.3:
            occupancy_class.append(0)
        else:
            occupancy_class.append(1)
    data["occupancy_class"] = occupancy_class
    
    # Show occupancy split
    print(data.occupancy_class.value_counts())
    
    return data

**Drop irrelevant columns**

In [53]:
# Define function for dropping irrelevant columns

def feat_drop_cols(data):
    
    # Drop further columns
    data.drop(
        [
            "active_months",
            "amenities",
            "am_coffee_machine",
            "am_cooking_basics",
            "am_parking",
            "availability_365",
            "avg_nights",
            "bathrooms",
            "beds",
            "bookings_est",
            "calculated_host_listings_count",
            "cleaning_fee",
            "descr_detail",
            "description",
            "description_exist",
            "description_len",
            "description_exist",
            "extra_people",
            "first_review",
            "first_review_days",
            "guests_included",
            "host_identity_verified",
            "house_rules",
            "house_rules_exist",
            "house_rules_len",
            "interaction",
            "interaction_exist",
            "interaction_len",
            "is_location_exact",
            "last_review",
            "last_review_days",
            "listing_url",
            "minimum_nights",
            "monthly_price",
            "name",
            "neighborhood_overview",
            "neighborhood_overview_exist",
            "neighborhood_overview_len",
            "notes",
            "notes_exist",
            "notes_len",
            "number_of_reviews",
            "number_of_reviews_ltm",
            #        "occupancy_class",
            #        "price",
            "price_calc",
            #        "price_avg", "price_diff", "price_diff_perc",
            "price_extra_fees",
            'review_scores_accuracy',
            'review_scores_checkin',
            'review_scores_cleanliness',
            'review_scores_communication',
            "review_scores_rating",
            'review_scores_value',
            "reviews_3mth",
            "security_deposit",
            "space",
            "space_exist",
            "space_len",
            "summary",
            "summary_exist",
            "summary_len",
            "text_len",
            "transit",
            "transit_exist",
            "transit_len",
            "weekly_price"
        ],
        inplace=True,
        axis=1)
        
    return data

A large number of features are dropped - some as they have been replaced, some have been deemed irrelevant, some have turned out to be highly correlated with others during EDA. Just a few notes on specific features:

| **DROPPED FEATURE** | **REASONING** |
| :----- | :----- |
| **am_coffee_machine** | high correlation (>0.3) with >5 other features |
| **am_parking** | high correlation (>0.3) with >5 other features |
| **availability_365** | high correlation (>0.3) with >5 other features |
| **descr_detail** | dropped in favour of **text_len** |
| **review_scores_xxx** | high correlation with review_scores_rating |


## Apply feature engineering functions

In [54]:
# Bundle feature engineering steps as function "feature_engineering"
def feature_engineering(data):
    data = feat_adapt(data)
    data = feat_bin(data)
    data = feat_num(data)
    data = feat_cat(data)
    data = feat_text(data)
    data = feat_log_sqrt(data)
    data = feat_occupancy(data)  
    data = feat_drop_cols(data)  
    return data

In [55]:
# Apply feature engineering to dataset
data = feature_engineering(data)

<ipython-input-46-4e73a752f4de>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["monthly_price"] = np.where(data.monthly_price == 0, data.price * 30,
<ipython-input-46-4e73a752f4de>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["weekly_price"] = np.where(data.weekly_price == 0, data.price * 7,
<ipython-input-46-4e73a752f4de>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

C:\Users\Mauricio\anaconda3\envs\airbnb-env\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-48-bde3733abde9>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["last_review_days"] = (datetime.strptime(dataset_date,
<ipython-input-48-bde3733abde9>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

0    7473
1    4295
Name: occupancy_class, dtype: int64


## Final check and export

In [56]:
# Sort columns in dataset
data = data.reindex(sorted(data.columns, reverse=False), axis=1)

In [57]:
# Review datatypes (data.info()) (post-engineering)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11768 entries, 23197 to 42833940
Data columns (total 51 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   accommodates                  11768 non-null  int64  
 1   accommodates_per_bed          11768 non-null  float64
 2   am_balcony                    11768 non-null  float64
 3   am_breakfast                  11768 non-null  float64
 4   am_child_friendly             11768 non-null  float64
 5   am_elevator                   11768 non-null  float64
 6   am_essentials                 11768 non-null  float64
 7   am_nature_and_views           11768 non-null  float64
 8   am_pets_allowed               11768 non-null  float64
 9   am_private_entrance           11768 non-null  float64
 10  am_smoking_allowed            11768 non-null  float64
 11  am_tv                         11768 non-null  float64
 12  am_white_goods                11768 non-null  float64

In [58]:
# Display engineered dataset
print(data.shape)
data.head(3)

(11768, 51)


,accommodates,accommodates_per_bed,am_balcony,am_breakfast,am_child_friendly,am_elevator,am_essentials,am_nature_and_views,am_pets_allowed,am_private_entrance,am_smoking_allowed,am_tv,am_white_goods,availability_90,bathrooms_log,bedrooms,calc_host_lst_count_sqrt_log,cancellation_policy,first_review_days_sqrt,guests_included_calc,host_acceptance_rate,host_is_superhost,host_response_rate,host_response_time,instant_bookable,last_review_days_sqrt,latitude,listing_no,longitude,maximum_nights,minimum_nights_sqrt,neighbourhood_cleansed,number_of_reviews_ltm_log,occupancy_class,occupancy_rate,price,price_calc_log,price_class,price_extra_fees_sqrt,price_extra_people,price_log,property_type,review_scores_calc,review_scores_class,review_scores_class_new,review_scores_location,review_scores_rating_sqrt,room_type,text_len_sqrt,wk_mth_discount,zipcode
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23197,6,1.2,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,85,0.693147,3.0,0.346574,strict,57.349804,3,100.0,1,100.0,within an hour,1,9.591663,41.41291,23197,2.22063,30,1.732051,el Besòs i el Maresme,4.123106,0,0.000000,197.19,5.232124,11,19.493589,8.0,5.284168,Apartment,93.643534,3,3,9.0,6.708204,Entire home/apt,0.845102,-0.115675,zip_other
25786,2,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,52,0.000000,1.0,0.000000,strict,59.203040,1,100.0,1,100.0,within an hour,1,1.000000,41.40145,25786,2.15645,730,1.000000,la Vila de Gràcia,7.071068,1,0.333333,37.64,3.628067,3,0.000000,25.0,3.628067,Apartment,94.858579,3,3,10.0,6.708204,Private room,0.588981,-0.115834,zip_00801
31958,4,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,52,0.000000,1.0,1.700599,strict,57.558666,4,100.0,0,91.0,within an hour,1,8.306624,41.40950,31958,2.15938,27,1.000000,el Camp d'en Grassot i Gràcia Nova,5.477226,0,0.066667,53.78,3.984902,5,16.062378,0.0,3.984902,Apartment,89.825266,2,2,9.0,5.944272,Entire home/apt,0.784311,-0.115656,zip_00802


**Export data_engineered**

In [59]:
# Export dataset for further use in 4_EDA_Engineered and 5_Predictive_Modeling
data_engineered = data.copy()
save_load(data_engineered, title="data_engineered", function="save")

# Preprocessing (train/test split and pipeline)

In [60]:
# Import data_engineered
data = save_load(title="data_engineered", function="load")

In [61]:
# Feature selection

#... by removing certain features
all_features = [
    el for el in data.columns if el not in [
        'occupancy_rate', 'occupancy_class', 'listing_no', 'price_log',
        'price_class', 'price_binary', "review_scores_class_new",
        "review_scores_class", "review_scores_calc", "neighbourhood_cleansed"
    ]
]

#... by only considering certain features
key_features = [
    "accommodates_per_bed", "am_balcony", "am_breakfast", "am_child_friendly",
    "am_elevator", "am_essentials", "am_pets_allowed", "am_private_entrance",
    "am_smoking_allowed", "am_tv", "bathrooms_log", "bedrooms",
    "calc_host_lst_count_sqrt_log", "cancellation_policy",
    "guests_included_calc", "host_is_superhost", "instant_bookable",
    "maximum_nights", "minimum_nights_sqrt", "property_type", "room_type",
    "wk_mth_discount", "zipcode"
]

# select features for predictive modeling from above: [all_features, key_features]
pred_features = key_features

#Display columns:
#all_features
#key_features

Please make sure to carefully select the features you want to include in modeling via the cell above. Below you will see the output and potential issues with the selection, if detected.

In [62]:
# Print target setting and feature selection
print_target_setting()

You are currently using PRICE_LOG as the target and neg_median_absolute_error for scoring to predict prices for barcelona on 2020-03-16

You are currently using these features for its prediction:
['accommodates_per_bed', 'am_balcony', 'am_breakfast', 'am_child_friendly', 'am_elevator', 'am_essentials', 'am_pets_allowed', 'am_private_entrance', 'am_smoking_allowed', 'am_tv', 'bathrooms_log', 'bedrooms', 'calc_host_lst_count_sqrt_log', 'cancellation_policy', 'guests_included_calc', 'host_is_superhost', 'instant_bookable', 'maximum_nights', 'minimum_nights_sqrt', 'property_type', 'room_type', 'wk_mth_discount', 'zipcode']

No issues with your selection of pred_features have been detected. Please make sure to manually check for correctness nevertheless.


In [63]:
# Drop columns
drop_columns = [el for el in data.columns if el not in pred_features]
drop_columns.remove(target)
data.drop(labels=drop_columns, inplace=True, axis=1)

In [64]:
# Drop rows (optional, just temporary)
#data = data[data.number_of_reviews_ltm_log>1.7]

## Preprocessing pipeline

In [65]:
# Create list for categorical predictors/features (used in "Scaling with Preprocessing Pipeline")
cat_features = list(data.columns[data.dtypes == object])
#cat_features.remove("neighbourhood")
#cat_features.remove("zipcode")
cat_features

['cancellation_policy', 'property_type', 'room_type', 'zipcode']

In [66]:
# Create list for numerical predictors/features (removing target column, used in "Scaling with Preprocessing Pipeline")
num_features = list(data.columns[data.dtypes != object])
num_features.remove(target)
num_features

['accommodates_per_bed',
 'am_balcony',
 'am_breakfast',
 'am_child_friendly',
 'am_elevator',
 'am_essentials',
 'am_pets_allowed',
 'am_private_entrance',
 'am_smoking_allowed',
 'am_tv',
 'bathrooms_log',
 'bedrooms',
 'calc_host_lst_count_sqrt_log',
 'guests_included_calc',
 'host_is_superhost',
 'instant_bookable',
 'maximum_nights',
 'minimum_nights_sqrt',
 'wk_mth_discount']

In [67]:
# Build preprocessor pipeline
# Pipeline for numerical features
num_pipeline = Pipeline([('imputer_num', SimpleImputer(strategy='median')),
                         ('std_scaler', StandardScaler())])

# Pipeline for categorical features
cat_pipeline = Pipeline([
    ('imputer_cat', SimpleImputer(strategy='constant', fill_value='missing')),
    ('1hot', OneHotEncoder(drop='first', handle_unknown='error'))
])

# Complete pipeline
preprocessor = ColumnTransformer([('num', num_pipeline, num_features),
                                  ('cat', cat_pipeline, cat_features)])

## Train/test split

In [68]:
# Define predictors and target variable
X = data.drop([target], axis=1)
y = data[target]

In [69]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    shuffle=True)
#                                                   stratify=y) # Use stratify=y if labels are inbalanced (e.g. most wines are 5 or 6; check with value_counts()!)

In [70]:
# Saving preprocessed X_train and X_test
X_train_prep_preprocessor = preprocessor.fit(X_train)

X_train_prep = X_train_prep_preprocessor.transform(X_train)
X_train_num_prep = num_pipeline.fit_transform(X_train[num_features])
X_test_prep = X_train_prep_preprocessor.transform(X_test)

In [71]:
# Get feature names from pipeline after one-hot encoding as "column_names"
onehot_columns = list(preprocessor.named_transformers_['cat']['1hot'].get_feature_names(cat_features))
column_names = num_features + onehot_columns

## Save preprocessor and X_test

In [72]:
X_train_prep

array([[-0.7906736 ,  1.35042041, -0.25481783, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.46151724, -0.74051014, -0.25481783, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.46151724, -0.74051014, -0.25481783, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.46151724, -0.74051014, -0.25481783, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.46151724, -0.74051014, -0.25481783, ...,  0.        ,
         0.        ,  0.        ],
       [-0.06022894, -0.74051014, -0.25481783, ...,  0.        ,
         0.        ,  0.        ]])

In [73]:
# Save preprocessor
save_load(X_train_prep_preprocessor, title="preprocessor", function="save")

In [74]:
# Save X_test
save_load(X_test, title="X_test", function="save")

## Feature selection (optional, on classification)

In [75]:
# Calculate "y_sel" from "price_log" (y) to get classification task
price_feat_sel = []
for price in y:
    if price <= 3.4:
        price_feat_sel.append(0)
    elif price <= 3.7:
        price_feat_sel.append(1)
    elif price <= 4:
        price_feat_sel.append(2)
    elif price <= 4.3:
        price_feat_sel.append(3)
    elif price <= 4.6:
        price_feat_sel.append(4)
    elif price <= 4.9:
        price_feat_sel.append(5)
    else:
        price_feat_sel.append(6)
y_sel = price_feat_sel

**Pearson's correlation**

In [76]:
# Define function
def cor_selector(X, y_sel):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in num_features:
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in num_features]
    return cor_support, cor_feature

In [77]:
# Execute function
cor_support, cor_feature = cor_selector(X, y_sel)
print(str(len(cor_feature)), 'selected features')

19 selected features


**Chi-squared**

In [78]:
# Perform chi-squared
X_norm = MinMaxScaler().fit_transform(X[num_features])
chi_selector = SelectKBest(chi2, k=10)
chi_selector.fit(X_norm, y_sel)
chi_support = chi_selector.get_support()
chi_feature = X[num_features].loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

10 selected features


**Recursive feature elimination**

In [79]:
# Perform RFE
rfe_selector = RFE(estimator=LogisticRegression(max_iter=3000), n_features_to_select=10, step=10, verbose=5)
rfe_selector.fit(X_norm, y_sel)
rfe_support = rfe_selector.get_support()
rfe_feature = X[num_features].loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 19 features.
10 selected features


**SelectFromModel: Lasso**

In [80]:
# Fit model
embedded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2", max_iter=3000), max_features=10)
embedded_lr_selector.fit(X_norm, y_sel)

SelectFromModel(estimator=LogisticRegression(max_iter=3000), max_features=10)

In [81]:
# Evaluate features
embedded_lr_support = embedded_lr_selector.get_support()
embedded_lr_feature = X[num_features].loc[:,embedded_lr_support].columns.tolist()
print(str(len(embedded_lr_feature)), 'selected features')

6 selected features


**SelectFromModel: Tree-based**

In [82]:
# Fit model
embedded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=10)
embedded_rf_selector = embedded_rf_selector.fit(X_norm, y_sel)

In [83]:
# Evaluate features
embedded_rf_support = embedded_rf_selector.get_support()
embedded_rf_feature = X[num_features].loc[:,embedded_rf_support].columns.tolist()
print(str(len(embedded_rf_feature)), 'selected features')

6 selected features


**SelectFromModel: LightGBM**

In [84]:
# Fit model
lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embedded_lgb_selector = SelectFromModel(lgbc, max_features=10)
embedded_lgb_selector = embedded_lgb_selector.fit(X_norm, y_sel)

In [85]:
# Evaluate features
embedded_lgb_support = embedded_lgb_selector.get_support()
embedded_lgb_feature = X[num_features].loc[:,embedded_lgb_support].columns.tolist()
print(str(len(embedded_lgb_feature)), 'selected features')

6 selected features


**Feature evaluation**

In [86]:
# Put all selections together
feature_selection_df = pd.DataFrame({'Feature':num_features, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgb_support})

In [87]:
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)

In [88]:
# display the top features
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(15)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,guests_included_calc,True,True,True,True,True,True,6
2,wk_mth_discount,False,True,True,True,True,True,5
3,minimum_nights_sqrt,True,False,True,True,True,True,5
4,calc_host_lst_count_sqrt_log,True,True,True,False,True,True,5
5,accommodates_per_bed,True,False,True,True,True,True,5
6,bedrooms,True,True,True,True,False,False,4
7,maximum_nights,True,False,False,False,True,True,3
8,bathrooms_log,True,False,True,True,False,False,3
9,am_tv,True,True,True,False,False,False,3
10,am_smoking_allowed,True,True,True,False,False,False,3


# Modeling: Regression ("price_log")

## Apply regression models

In [89]:
# Print current setting for TARGET
print_target_setting()

You are currently using PRICE_LOG as the target and neg_median_absolute_error for scoring to predict prices for barcelona on 2020-03-16

You are currently using these features for its prediction:
['accommodates_per_bed', 'am_balcony', 'am_breakfast', 'am_child_friendly', 'am_elevator', 'am_essentials', 'am_pets_allowed', 'am_private_entrance', 'am_smoking_allowed', 'am_tv', 'bathrooms_log', 'bedrooms', 'calc_host_lst_count_sqrt_log', 'cancellation_policy', 'guests_included_calc', 'host_is_superhost', 'instant_bookable', 'maximum_nights', 'minimum_nights_sqrt', 'property_type', 'room_type', 'wk_mth_discount', 'zipcode']

No issues with your selection of pred_features have been detected. Please make sure to manually check for correctness nevertheless.


In [90]:
# Select models for comparison
regmodels = {
    'Baseline':
    DummyRegressor(strategy='mean'),
    'LinReg':
    LinearRegression(),
    'Passive Aggressive':
    PassiveAggressiveRegressor(),
    #        'RANSAC' : RANSACRegressor(),
    'ElasticNet':
    ElasticNet(),
    'Stochastic Gradient Descent':
    SGDRegressor(max_iter=1000, tol=1e-3),
    'Decision Tree':
    DecisionTreeRegressor(criterion="mse",
                          max_depth=3,
                          random_state=random_state),
    'Random Forest':
    RandomForestRegressor(random_state=random_state,
                          max_features='sqrt',
                          n_jobs=-1),
    'Gradient Boost':
    GradientBoostingRegressor(random_state=random_state),
    'XGBoost':
    XGBRegressor(),
    'AdaBoost':
    AdaBoostRegressor(random_state=random_state),
    'SVR':
    SVR(),
    'CatBoost':
    CatBoostRegressor()
}

In [91]:
# Calculate and display results
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'R2', 'MAE', 'MAPE', 'MAPE median'])
i = 0
for m in regmodels.items():
    # Building a full pipeline with our preprocessor and a Classifier
    pipe = Pipeline([('preprocessor', preprocessor), (m[0], m[1])])
    # Making predictions on the training set using cross validation as well as calculating the probabilities
    y_train_pred = cross_val_predict(pipe,
                                     X_train,
                                     y_train.values.ravel(),
                                     cv=5,
                                     verbose=4,
                                     n_jobs=-1)
    # Calculating metrices
    temp = pd.DataFrame(
        {
            'Model':
            m[0],
            'MSE':
            "{:.2f}".format(mean_squared_error(y_train, y_train_pred)),
            'RMSE':
            "{:.2f}".format(
                mean_squared_error(y_train, y_train_pred, squared=False)),
            'R2':
            "{:.2f}".format(r2_score(y_train, y_train_pred)),
            'MAE':
            "{:.2f}".format(mean_absolute_error(y_train, y_train_pred)),
            'MAPE': mean_absolute_percentage_error(y_train, y_train_pred),
            'MAPE median': median_absolute_percentage_error(y_train, y_train_pred)
        },
        index=[i])
    i += 1
    results = pd.concat([results, temp])
results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.4s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_

,Model,MSE,RMSE,R2,MAE,MAPE,MAPE median
0,Baseline,0.52,0.72,-0.00,0.59,15.083082,12.963619
1,LinReg,0.22,0.47,0.57,0.37,9.290625,7.442362
2,Passive Aggressive,0.73,0.86,-0.42,0.55,14.194402,11.150733
3,ElasticNet,0.52,0.72,-0.00,0.59,15.083082,12.963619
4,Stochastic Gradient Descent,19909.96,141.10,-38633.00,112.84,2864.186948,2776.533058
5,Decision Tree,0.22,0.46,0.58,0.36,9.075547,7.188912
6,Random Forest,0.11,0.33,0.79,0.25,6.249740,4.631559
7,Gradient Boost,0.12,0.35,0.77,0.27,6.687716,5.280145
8,XGBoost,0.09,0.29,0.83,0.21,5.260144,3.708981
9,AdaBoost,0.17,0.41,0.68,0.33,8.309028,6.948571


## Reg model 1: XGBoost

In [92]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_xgb_reg = Pipeline([('preprocessor', preprocessor),
                             ('xgb_reg',
                              XGBRegressor(n_estimators=182,
                                           learning_rate=0.45,
                                           random_state=random_state,
                                           max_depth=5,
                                           gamma=0.3,
                                           bootstrap=True,
                                           max_features=21,
                                           scoring=scoring,
                                           n_jobs=-1))])

### Hyperparameter pre-tuning with RandomizedSearchCV

In [93]:
# Display possible hyperparameters for XGBoost Regressor
test_xgb_reg = XGBRegressor()
test_xgb_reg.get_params().keys()

dict_keys(['objective', 'base_score', 'booster', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'gamma', 'gpu_id', 'importance_type', 'interaction_constraints', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'monotone_constraints', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'subsample', 'tree_method', 'validate_parameters', 'verbosity'])

**Default values for XGBRegressor** (as base for hyperparameter search):

max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective='reg:linear', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, importance_type='gain'

In [94]:
# Define hyperparameter distribution
param_distribs_xgb_reg = {
    'xgb_reg__n_estimators': randint(low=80, high=300),
    'xgb_reg__bootstrap': [True, False],
    'xgb_reg__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'xgb_reg__max_depth': randint(low=1, high=7),
    'xgb_reg__max_features': randint(low=1, high=40),
    'xgb_reg__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
}

In [95]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_xgb_reg = RandomizedSearchCV(pipeline_xgb_reg,
                                 param_distribs_xgb_reg,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=10,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_xgb_reg = rnd_xgb_reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   26.1s finished


[10:46:45] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { bootstrap, max_features, scoring } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [96]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_xgb_reg.best_score_))
print("Best parameters:\n{}".format(rnd_xgb_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_xgb_reg.best_estimator_))

Best score:
-0.17
Best parameters:
{'xgb_reg__bootstrap': True, 'xgb_reg__gamma': 0.3, 'xgb_reg__learning_rate': 0.45, 'xgb_reg__max_depth': 5, 'xgb_reg__max_features': 21, 'xgb_reg__n_estimators': 182}


### Hyperparameter tuning with GridSearchCV

In [97]:
# Define hyperparameter grid
param_grid_xgb_reg = {
#    'xgb_reg__bootstrap': [True, False],
#    'xgb_reg__n_estimators': [190, 230, 290],
#    'xgb_reg__max_features': [40, 45],
    'xgb_reg__max_depth': [4, 5],
    'xgb_reg__learning_rate': [0.42, 0.45, 0.48]
}

In [98]:
# Create and fit GridSearchCV, save "best_model"
grid_xgb_reg = GridSearchCV(pipeline_xgb_reg,
                            param_grid_xgb_reg,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_xgb_reg = grid_xgb_reg.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   17.8s finished


[10:47:04] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { bootstrap, max_features, scoring } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [99]:
# Assign result to best model
best_model_xgb_reg = grid_xgb_reg.best_estimator_['xgb_reg']

In [100]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_xgb_reg.best_score_))
print("Best parameters:\n{}".format(grid_xgb_reg.best_params_))
#print("Best estimator:\n{}".format(grid_xgb_reg.best_estimator_))

Best score:
-0.17
Best parameters:
{'xgb_reg__learning_rate': 0.45, 'xgb_reg__max_depth': 5}


### Feature importances

In [101]:
# Display feature importances
fi_xgb_reg = get_feat_importances(best_model_xgb_reg, column_names=column_names)
fi_xgb_reg

,weight
room_type_Private room,0.513661
bedrooms,0.124733
guests_included_calc,0.055987
room_type_Shared room,0.048753
minimum_nights_sqrt,0.031138
wk_mth_discount,0.023839
cancellation_policy_super_strict,0.021867
property_type_Boutique hotel,0.017382
bathrooms_log,0.011595
am_elevator,0.010376


### Final evaluation best model

In [102]:
# Load existing model
#best_model_xgb_reg = save_load(title="best_model_xgb_reg", function="load")
#load_best_cv = save_load(title="best_cv_xgb_reg", function="load")

**Learning Curves (Overfitting)**

**Training Set**

In [103]:
# Predict target with "best model"
y_train_pred_xgb_reg = best_model_xgb_reg.predict(X_train_prep)

In [104]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_xgb_reg, model="reg")

MSE: 0.06
RMSE: 0.25
MAE: 0.19
R2: 0.88
MAPE: 4.63
MAPE median: 3.42


**Testing Set**

In [105]:
# Predict target with "best model"
y_test_pred_xgb_reg = best_model_xgb_reg.predict(X_test_prep)

In [106]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_xgb_reg, model="reg")

MSE: 0.09
RMSE: 0.30
MAE: 0.21
R2: 0.83
MAPE: 5.37
MAPE median: 3.85


In [107]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_xgb_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.28229273, 0.30721243])

**Median Price Intervals**

In [108]:
# Save MAPE_median as variable
MAPE_median_xgb_reg = (median_absolute_percentage_error(y_test, y_test_pred_xgb_reg))/100

In [109]:
# Calculate price interval for MAPE median
y_pred_interval_xgb_reg = tuple([(round(math.exp(el-el*MAPE_median_xgb_reg),2),round(math.exp(el+el*MAPE_median_xgb_reg),2)) for el in y_test_pred_xgb_reg])
y_pred_interval_xgb_reg

((38.62, 51.73),
 (54.24, 74.65),
 (85.0, 121.28),
 (38.74, 51.9),
 (67.34, 94.3),
 (23.13, 29.74),
 (44.28, 59.96),
 (63.51, 88.52),
 (53.63, 73.75),
 (87.7, 125.43),
 (35.76, 47.61),
 (36.99, 49.38),
 (53.41, 73.42),
 (33.45, 44.29),
 (15.77, 19.67),
 (109.69, 159.73),
 (33.13, 43.84),
 (43.89, 59.39),
 (75.6, 106.86),
 (76.66, 108.48),
 (15.99, 19.97),
 (36.52, 48.7),
 (119.17, 174.68),
 (18.53, 23.41),
 (45.3, 61.46),
 (58.87, 81.57),
 (41.99, 56.63),
 (86.25, 123.2),
 (32.34, 42.71),
 (101.44, 146.8),
 (145.31, 216.4),
 (62.74, 87.37),
 (179.94, 272.6),
 (29.04, 38.03),
 (24.45, 31.57),
 (53.8, 74.0),
 (128.16, 188.96),
 (28.07, 36.66),
 (117.11, 171.42),
 (111.49, 162.56),
 (234.29, 362.51),
 (80.4, 114.21),
 (29.4, 38.52),
 (79.21, 112.38),
 (206.13, 315.69),
 (28.55, 37.33),
 (145.37, 216.51),
 (49.54, 67.69),
 (31.54, 41.57),
 (29.87, 39.2),
 (39.16, 52.51),
 (74.95, 105.87),
 (67.33, 94.29),
 (18.27, 23.06),
 (29.21, 38.26),
 (32.74, 43.27),
 (20.46, 26.05),
 (89.16, 127.7),


**Save Model and Params**

In [110]:
# Save best model and cv
save_load(best_model_xgb_reg, title="best_model_xgb_reg", function="save")
save_load(grid_xgb_reg, title="best_cv_xgb_reg", function="save")

## Reg model 2: Support Vector Machines

In [111]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_svm_reg = Pipeline([('preprocessor', preprocessor),
                             ('svm_reg',
                              SVR(kernel='rbf',
                                  C=50,
                                  degree=4,
                                  gamma=0.005,
                                  epsilon=0.3))])

### Hyperparameter pre-tuning with RandomizedSearchCV

In [112]:
# Display possible hyperparameters for Support Vector Machine
test_svr_reg = SVR()
test_svr_reg.get_params().keys()

dict_keys(['C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', 'tol', 'verbose'])

**Default values for Support Vector Machine** (as base for hyperparameter search):

kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1

In [113]:
# Define hyperparameter distribution
param_distribs_svm_reg = {
#    'svm_reg__kernel': ['linear', 'poly', 'rbf'],
    'svm_reg__C': [0.1, 0.5, 0.8, 1, 1.5, 2, 3, 5, 10, 50, 100],        # initial: [0.1, 0.5, 1, 2, 5, 10, 50, 100, 500, 1000]
    'svm_reg__gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1],   # initial: [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1]
    'svm_reg__epsilon': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.9],            # initial: [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.9]
    'svm_reg__degree': randint(low=1, high=5)
}

In [114]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_svm_reg = RandomizedSearchCV(pipeline_svm_reg,
                                 param_distribs_svm_reg,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=10,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_svm_reg = rnd_svm_reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   43.6s finished


In [115]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_svm_reg.best_score_))
print("Best parameters:\n{}".format(rnd_svm_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_svm_reg.best_estimator_))

Best score:
-0.21
Best parameters:
{'svm_reg__C': 10, 'svm_reg__degree': 1, 'svm_reg__epsilon': 0.2, 'svm_reg__gamma': 0.001}


### Hyperparameter tuning with GridSearchCV

In [116]:
# Define hyperparameter grid
param_grid_svm_reg = {
#    'svm_reg__kernel': ['linear', 'poly', 'rbf'],
    'svm_reg__gamma': [0.003, 0.005, 0.007],
    'svm_reg__C': [40, 50, 60],
    'svm_reg__degree': [3, 4, 5]
}

In [117]:
# Create and fit GridSearchCV, save "best_model"
grid_svm_reg = GridSearchCV(pipeline_svm_reg,
                            param_grid_svm_reg,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_svm_reg = grid_svm_reg.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  2.1min finished


In [118]:
# Assign result to best model
best_model_svm_reg = grid_svm_reg.best_estimator_["svm_reg"]

In [119]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_svm_reg.best_score_))
print("Best parameters:\n{}".format(grid_svm_reg.best_params_))
#print("Best estimator:\n{}".format(grid_svm_reg.best_estimator_))

Best score:
-0.21
Best parameters:
{'svm_reg__C': 60, 'svm_reg__degree': 3, 'svm_reg__gamma': 0.003}


### Feature importances

In [120]:
# Display feature importances
#fi_svm_reg = get_feat_importances(best_model_svm_reg, column_names=column_names)
#fi_svm_reg

### Final evaluation best model

In [121]:
# Load existing model
#load_best_model = load_model(title="best_model_svm_reg_01", dataset_loc=dataset_loc, dataset_date=dataset_date)
#load_best_cv = load_model(title="best_cv_svm_reg_01", dataset_loc=dataset_loc, dataset_date=dataset_date)

**Learning Curves (Overfitting)**

**Training Set**

In [122]:
# Predict target with "best model"
y_train_pred_svm_reg = best_model_svm_reg.predict(X_train_prep)

In [123]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_svm_reg, model="reg")

MSE: 0.09
RMSE: 0.30
MAE: 0.23
R2: 0.63
MAPE: 4.93
MAPE median: 4.33


**Testing Set**

In [124]:
# Predict target with "best model"
y_test_pred_svm_reg = best_model_svm_reg.predict(X_test_prep)

In [125]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_svm_reg, model="reg")

MSE: 0.11
RMSE: 0.32
MAE: 0.25
R2: 0.54
MAPE: 5.30
MAPE median: 4.44


In [126]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_svm_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.31179346, 0.33705304])

**Median Price Intervals**

In [127]:
# Save MAPE_median as variable
MAPE_median_svm_reg = (median_absolute_percentage_error(y_test, y_test_pred_svm_reg))/100

In [128]:
# Calculate price interval for MAPE median
y_pred_interval_svm_reg = tuple([(round(math.exp(el-el*MAPE_median_svm_reg),2),round(math.exp(el+el*MAPE_median_svm_reg),2)) for el in y_test_pred_svm_reg])
y_pred_interval_svm_reg

((68.31, 101.11),
 (124.58, 194.98),
 (142.68, 226.14),
 (141.88, 224.75),
 (233.24, 386.91),
 (112.1, 173.74),
 (90.73, 137.89),
 (111.3, 172.38),
 (103.56, 159.32),
 (80.24, 120.56),
 (93.29, 142.15),
 (262.65, 440.55),
 (94.77, 144.61),
 (93.32, 142.19),
 (47.69, 68.27),
 (123.41, 192.98),
 (76.67, 114.7),
 (74.93, 111.86),
 (110.11, 170.37),
 (66.84, 98.74),
 (115.83, 180.07),
 (100.84, 154.75),
 (91.32, 138.86),
 (134.42, 211.86),
 (73.78, 109.99),
 (138.12, 218.24),
 (55.67, 80.85),
 (105.76, 163.03),
 (43.54, 61.8),
 (99.13, 151.88),
 (68.26, 101.03),
 (108.29, 167.3),
 (97.57, 149.27),
 (79.28, 118.98),
 (129.35, 203.15),
 (107.66, 166.22),
 (90.01, 136.69),
 (105.46, 162.51),
 (111.49, 172.71),
 (79.98, 120.12),
 (94.53, 144.2),
 (137.42, 217.04),
 (76.4, 114.26),
 (90.89, 138.15),
 (88.28, 133.82),
 (91.6, 139.32),
 (61.0, 89.34),
 (113.36, 175.86),
 (66.95, 98.91),
 (61.46, 90.09),
 (56.88, 82.77),
 (56.29, 81.84),
 (98.34, 150.56),
 (107.48, 165.92),
 (156.15, 249.57),
 (10

**Save Model and Params**

In [129]:
# Save best model and cv
save_load(best_model_svm_reg, title="best_model_svm_reg", function="save")
save_load(grid_svm_reg, title="best_cv_svm_reg", function="save")

## Reg model 3: Random Forest

In [130]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_rf_reg = Pipeline([('preprocessor', preprocessor),
                            ('rf_reg',
                             RandomForestRegressor(n_estimators=1500,
                                                   max_features='sqrt',
                                                   random_state=random_state,
                                                   max_depth=4,
                                                   min_samples_split=10,
                                                   min_samples_leaf=1,
                                                   bootstrap=False,
                                                   n_jobs=-1))])

### Hyperparameter pre-tuning with RandomizedSearchCV

In [131]:
# Display possible hyperparameters for XGBoost Regressor
test_rf_reg = RandomForestRegressor()
test_rf_reg.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

**Default values for Random Forest Regressor** (as base for hyperparameter search):

n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None

In [132]:
# Define hyperparameter distribution
param_distribs_rf_reg = {
    'rf_reg__n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'rf_reg__max_features': ['auto', 'sqrt'],
    'rf_reg__max_depth': randint(low=1, high=15),
    'rf_reg__min_samples_split': randint(low=1, high=10),
    'rf_reg__min_samples_leaf': randint(low=1, high=10),
    'rf_reg__bootstrap': [True, False]
}

In [133]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_rf_reg = RandomizedSearchCV(pipeline_rf_reg,
                                 param_distribs_rf_reg,
                                 cv=4,
                                 scoring=scoring,
                                 n_iter=5,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

rnd_rf_reg = rnd_rf_reg.fit(X_train, y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.0min finished


In [134]:
# Assign result to best model
best_model_rnd_rf_reg = rnd_rf_reg.best_estimator_['rf_reg']

In [135]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_rf_reg.best_score_))
print("Best parameters:\n{}".format(rnd_rf_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_rf_reg.best_estimator_))

Best score:
-0.18
Best parameters:
{'rf_reg__bootstrap': False, 'rf_reg__max_depth': 8, 'rf_reg__max_features': 'auto', 'rf_reg__min_samples_leaf': 6, 'rf_reg__min_samples_split': 5, 'rf_reg__n_estimators': 400}


### Hyperparameter tuning with GridSearchCV

In [136]:
# Define hyperparameter grid
#param_grid_rf_reg = {
#    'rf_reg__n_estimators': [1200, 2000],
#    'rf_reg__max_features': ['auto', 'sqrt'],
#    'rf_reg__max_depth': [10, 15],
#    'rf_reg__min_samples_split': [6, 10],
#    'rf_reg__min_samples_leaf': [1, 2],
#    'rf_reg__bootstrap': [True, False]
#}

In [137]:
# Create and fit GridSearchCV, save "best_model"
#grid_rf_reg = GridSearchCV(pipeline_rf_reg,
#                            param_grid_rf_reg,
#                            cv=5,
#                            scoring=scoring,
#                            return_train_score=True,
#                            verbose=4,
#                            n_jobs=-1)

#grid_rf_reg = grid_rf_reg.fit(X_train, y_train)

In [138]:
# Assign result to best model
#best_model_rf_reg = grid_rf_reg.best_estimator_['rf_reg']

In [139]:
# Display best_score_, best_params_ and best_estimator_
#print('Best score:\n{:.2f}'.format(grid_rf_reg.best_score_))
#print("Best parameters:\n{}".format(grid_rf_reg.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

### Feature importances

In [140]:
# Display feature importances
fi_rf_reg = get_feat_importances(best_model_rnd_rf_reg, column_names=column_names)
fi_rf_reg

,weight
bedrooms,0.374157
wk_mth_discount,0.337351
room_type_Private room,0.151992
guests_included_calc,0.072530
am_essentials,0.012988
zipcode_zip_1012,0.008699
am_private_entrance,0.007405
zipcode_zip_1017,0.004656
maximum_nights,0.003800
bathrooms_log,0.003713


### Final evaluation best model

In [141]:
# Load existing model
#load_best_model = save_load(title="best_model_rf_reg", function="load")
#load_best_cv = save_load(title="best_cv_rf_reg", function="load")

**Learning Curves (Overfitting)**

**Training Set**

In [142]:
# Predict target with "best model"
y_train_pred_rf_reg = best_model_rnd_rf_reg.predict(X_train_prep)

In [143]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_rf_reg, model="reg")

MSE: 0.08
RMSE: 0.29
MAE: 0.21
R2: 0.65
MAPE: 4.33
MAPE median: 3.25


**Testing Set**

In [144]:
# Predict target with "best model"
y_test_pred_rf_reg = best_model_rnd_rf_reg.predict(X_test_prep)

In [145]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_rf_reg, model="reg")

MSE: 0.10
RMSE: 0.31
MAE: 0.23
R2: 0.59
MAPE: 4.73
MAPE median: 3.65


In [146]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_rf_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.29445901, 0.32174325])

**Median Price Intervals**

In [147]:
# Save MAPE_median as variable
MAPE_median_rf_reg = (median_absolute_percentage_error(y_test, y_test_pred_rf_reg))/100

In [148]:
# Calculate price interval for MAPE median
y_pred_interval_rf_reg = tuple([(round(math.exp(el-el*MAPE_median_rf_reg),2),round(math.exp(el+el*MAPE_median_rf_reg),2)) for el in y_test_pred_rf_reg])
y_pred_interval_rf_reg

((125.95, 181.7),
 (71.41, 98.68),
 (197.59, 294.94),
 (199.54, 298.07),
 (173.39, 256.26),
 (125.95, 181.7),
 (125.95, 181.7),
 (111.86, 159.93),
 (111.86, 159.93),
 (81.11, 113.17),
 (75.87, 105.32),
 (173.39, 256.26),
 (111.86, 159.93),
 (125.95, 181.7),
 (41.44, 54.95),
 (105.65, 150.4),
 (55.64, 75.45),
 (59.39, 80.93),
 (159.64, 234.47),
 (111.25, 158.99),
 (123.1, 177.28),
 (110.52, 157.86),
 (110.52, 157.86),
 (170.96, 252.4),
 (125.95, 181.7),
 (127.28, 183.76),
 (46.71, 62.5),
 (81.68, 114.03),
 (53.28, 72.0),
 (132.16, 191.35),
 (60.09, 81.96),
 (118.25, 169.78),
 (105.65, 150.4),
 (92.17, 129.86),
 (118.25, 169.78),
 (110.52, 157.86),
 (118.25, 169.78),
 (111.86, 159.93),
 (57.91, 78.77),
 (92.17, 129.86),
 (105.65, 150.4),
 (139.54, 202.87),
 (107.24, 152.84),
 (75.87, 105.32),
 (90.04, 126.62),
 (82.27, 114.92),
 (93.22, 131.44),
 (118.25, 169.78),
 (52.56, 70.96),
 (59.39, 80.93),
 (60.09, 81.96),
 (44.11, 58.77),
 (81.68, 114.03),
 (94.29, 133.07),
 (159.64, 234.47),
 (

**Save Model and Params**

In [149]:
# Save best model and cv
save_load(best_model_rnd_rf_reg, title="best_model_rf_reg", function="save")
save_load(rnd_rf_reg, title="best_cv_rf_reg", function="save")

## Reg model 4: CatBoost

In [150]:
# Create pipeline to use in RandomizedSearchCV and GridSearchCV
pipeline_cat_reg = Pipeline([('preprocessor', preprocessor),
                             ('cat_reg',
                              CatBoostRegressor(n_estimators=150,
                                                learning_rate=0.3,
                                                l2_leaf_reg=4,
                                                loss_function="RMSE",
                                                random_state=random_state,
                                                depth=4))])

### Hyperparameter pre-tuning with RandomizedSearchCV

In [151]:
# Display possible hyperparameters for XGBoost Regressor
test_cat_reg = CatBoostRegressor()
test_cat_reg.get_params().keys()

dict_keys(['loss_function'])

**Default values for CatBoostRegressor** (as base for hyperparameter search):

iterations=None, learning_rate=None, depth=None, l2_leaf_reg=None, model_size_reg=None, rsm=None, loss_function='RMSE', border_count=None, feature_border_type=None, per_float_feature_quantization=None, input_borders=None, output_borders=None, fold_permutation_block=None, od_pval=None, od_wait=None, od_type=None, nan_mode=None, counter_calc_method=None, leaf_estimation_iterations=None, leaf_estimation_method=None, thread_count=None, random_seed=None, use_best_model=None, best_model_min_trees=None, verbose=None, silent=None, logging_level=None, metric_period=None, ctr_leaf_count_limit=None, store_all_simple_ctr=None, max_ctr_complexity=None, has_time=None, allow_const_label=None, target_border=None, one_hot_max_size=None, random_strength=None, name=None, ignored_features=None, train_dir=None, custom_metric=None, eval_metric=None, bagging_temperature=None, save_snapshot=None, snapshot_file=None, snapshot_interval=None, fold_len_multiplier=None, used_ram_limit=None, gpu_ram_part=None, pinned_memory_size=None, allow_writing_files=None, final_ctr_computation_mode=None, approx_on_full_history=None, boosting_type=None, simple_ctr=None, combinations_ctr=None, per_feature_ctr=None, ctr_description=None, ctr_target_border_count=None, task_type=None, device_config=None, devices=None, bootstrap_type=None, subsample=None, mvs_reg=None, sampling_frequency=None, sampling_unit=None, dev_score_calc_obj_block_size=None, dev_efb_max_buckets=None, sparse_features_conflict_fraction=None, max_depth=None, n_estimators=None, num_boost_round=None, num_trees=None, colsample_bylevel=None, random_state=None, reg_lambda=None, objective=None, eta=None, max_bin=None, gpu_cat_features_storage=None, data_partition=None, metadata=None, early_stopping_rounds=None, cat_features=None, grow_policy=None, min_data_in_leaf=None, min_child_samples=None, max_leaves=None, num_leaves=None, score_function=None, leaf_estimation_backtracking=None, ctr_history_unit=None, monotone_constraints=None, feature_weights=None, penalties_coefficient=None, first_feature_use_penalties=None, per_object_feature_penalties=None, model_shrink_rate=None, model_shrink_mode=None, langevin=None, diffusion_temperature=None, boost_from_average=None

In [152]:
# Define hyperparameter distribution
param_distribs_cat_reg = {
    'cat_reg__n_estimators': randint(low=130, high=180),    # initial: randint(low=10, high=200)
    'cat_reg__l2_leaf_reg': randint(low=2, high=11),       # initial: randint(low=1, high=15)
    'cat_reg__depth': randint(low=4, high=6),             # initial: randint(low=1, high=15)
    'cat_reg__learning_rate': [0.15, 0.18, 0.2, 0.22, 0.25, 0.27, 0.3] # initial: [0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
}

In [153]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_cat_reg = RandomizedSearchCV(pipeline_cat_reg,
                                 param_distribs_cat_reg,
                                 cv=5,
                                 scoring=scoring,
                                 n_iter=15,
                                 return_train_score=True,
                                 verbose=4,
                                 n_jobs=-1,
                                 random_state=random_state)

best_model_rnd_cat_reg = rnd_cat_reg.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   40.5s finished


0:	learn: 0.4289827	total: 54.5ms	remaining: 8.12s
1:	learn: 0.3969317	total: 55.9ms	remaining: 4.14s
2:	learn: 0.3784942	total: 57.3ms	remaining: 2.81s
3:	learn: 0.3651901	total: 58.8ms	remaining: 2.15s
4:	learn: 0.3535714	total: 60.2ms	remaining: 1.75s
5:	learn: 0.3447556	total: 61.8ms	remaining: 1.48s
6:	learn: 0.3403435	total: 63.2ms	remaining: 1.29s
7:	learn: 0.3319494	total: 65.6ms	remaining: 1.16s
8:	learn: 0.3272539	total: 67.2ms	remaining: 1.05s
9:	learn: 0.3215778	total: 68.7ms	remaining: 962ms
10:	learn: 0.3199967	total: 70.1ms	remaining: 885ms
11:	learn: 0.3168571	total: 71.4ms	remaining: 821ms
12:	learn: 0.3152111	total: 72.8ms	remaining: 767ms
13:	learn: 0.3133584	total: 74.1ms	remaining: 720ms
14:	learn: 0.3121018	total: 75.4ms	remaining: 679ms
15:	learn: 0.3105061	total: 76.7ms	remaining: 642ms
16:	learn: 0.3092786	total: 79ms	remaining: 618ms
17:	learn: 0.3066250	total: 80.5ms	remaining: 590ms
18:	learn: 0.3054762	total: 81.8ms	remaining: 564ms
19:	learn: 0.3039280	tot

In [154]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_cat_reg.best_score_))
print("Best parameters:\n{}".format(rnd_cat_reg.best_params_))
#print("Best estimator:\n{}".format(rnd_cat_reg.best_estimator_))

Best score:
-0.16
Best parameters:
{'cat_reg__depth': 4, 'cat_reg__l2_leaf_reg': 4, 'cat_reg__learning_rate': 0.3, 'cat_reg__n_estimators': 150}


### Hyperparameter tuning with GridSearchCV

In [155]:
# Define hyperparameter grid
param_grid_cat_reg = {
#    'cat_reg__n_estimators': [150, 155],
#    'cat_reg__l2_leaf_reg': [3, 4],
    'cat_reg__depth': [4, 5],
#    'cat_reg__learning_rate': [0.15, 0.2, 0.25]
}

In [156]:
# Create and fit GridSearchCV, save "best_model"
grid_cat_reg = GridSearchCV(pipeline_cat_reg,
                            param_grid_cat_reg,
                            cv=5,
                            scoring=scoring,
                            return_train_score=True,
                            verbose=4,
                            n_jobs=-1)

grid_cat_reg = grid_cat_reg.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    3.3s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.5s finished


0:	learn: 0.4262223	total: 3.08ms	remaining: 458ms
1:	learn: 0.3879917	total: 5.13ms	remaining: 380ms
2:	learn: 0.3640841	total: 6.96ms	remaining: 341ms
3:	learn: 0.3511369	total: 8.78ms	remaining: 320ms
4:	learn: 0.3419162	total: 10.8ms	remaining: 313ms
5:	learn: 0.3327047	total: 13.2ms	remaining: 316ms
6:	learn: 0.3259296	total: 15ms	remaining: 306ms
7:	learn: 0.3219920	total: 16.9ms	remaining: 300ms
8:	learn: 0.3185234	total: 18.9ms	remaining: 296ms
9:	learn: 0.3142897	total: 20.8ms	remaining: 291ms
10:	learn: 0.3120262	total: 22.5ms	remaining: 284ms
11:	learn: 0.3103675	total: 24.3ms	remaining: 279ms
12:	learn: 0.3066015	total: 27.2ms	remaining: 287ms
13:	learn: 0.3051740	total: 29ms	remaining: 282ms
14:	learn: 0.3041403	total: 30.9ms	remaining: 278ms
15:	learn: 0.3028514	total: 32.7ms	remaining: 274ms
16:	learn: 0.3006763	total: 34.5ms	remaining: 270ms
17:	learn: 0.2995436	total: 36.1ms	remaining: 265ms
18:	learn: 0.2980029	total: 37.9ms	remaining: 261ms
19:	learn: 0.2972245	total

In [157]:
# Assign result to best model
best_model_cat_reg = grid_cat_reg.best_estimator_['cat_reg']

In [158]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_cat_reg.best_score_))
print("Best parameters:\n{}".format(grid_cat_reg.best_params_))
#print("Best estimator:\n{}".format(grid_xgb_reg.best_estimator_))

Best score:
-0.16
Best parameters:
{'cat_reg__depth': 5}


### Feature importances

In [159]:
# Display feature importances
fi_cat_reg = get_feat_importances(best_model_cat_reg, column_names=column_names)
fi_cat_reg

,weight
wk_mth_discount,40.950080
guests_included_calc,14.659045
room_type_Private room,11.414136
bedrooms,8.226997
am_tv,1.717044
minimum_nights_sqrt,1.618167
zipcode_zip_1012,1.518304
calc_host_lst_count_sqrt_log,1.380087
bathrooms_log,1.286109
zipcode_zip_1016,1.272065


### Final evaluation best model

In [160]:
# Load existing model
#load_best_model = save_load(title="best_model_cat_reg", function="load")
#load_best_cv = save_load(title="best_cv_cat_reg", function="load")

**Learning Curves (Overfitting)**

**Training Set**

In [161]:
# Predict target with "best model"
y_train_pred_cat_reg = best_model_cat_reg.predict(X_train_prep)

In [162]:
# Final evaluation of "best model"
model_eval(y_train, y_train_pred_cat_reg, model="reg")

MSE: 0.05
RMSE: 0.23
MAE: 0.18
R2: 0.78
MAPE: 3.72
MAPE median: 2.96


**Testing Set**

In [163]:
# Predict target with "best model"
y_test_pred_cat_reg = best_model_cat_reg.predict(X_test_prep)

In [164]:
# Final evaluation of "best model"
model_eval(y_test, y_test_pred_cat_reg, model="reg")

MSE: 0.07
RMSE: 0.26
MAE: 0.20
R2: 0.70
MAPE: 4.26
MAPE median: 3.46


In [165]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_test_pred_cat_reg - y_test)**2
np.sqrt(
    stats.t.interval(confidence,
                     len(squared_errors) - 1,
                     loc=squared_errors.mean(),
                     scale=stats.sem(squared_errors)))

array([0.25171763, 0.27433345])

**Median Price Intervals**

In [166]:
# Save MAPE_median as variable
MAPE_median_cat_reg = (median_absolute_percentage_error(y_test, y_test_pred_cat_reg))/100

In [167]:
# Calculate price interval for MAPE median
y_pred_interval_cat_reg = tuple([(round(math.exp(el-el*MAPE_median_cat_reg),2),round(math.exp(el+el*MAPE_median_cat_reg),2)) for el in y_test_pred_cat_reg])
y_pred_interval_cat_reg

((85.66, 117.83),
 (72.34, 98.32),
 (163.52, 235.61),
 (199.76, 291.99),
 (208.94, 306.39),
 (126.07, 178.3),
 (116.92, 164.47),
 (126.49, 178.93),
 (112.56, 157.9),
 (75.79, 103.35),
 (104.17, 145.32),
 (280.42, 419.97),
 (103.9, 144.93),
 (116.46, 163.77),
 (35.59, 45.98),
 (116.84, 164.35),
 (67.52, 91.31),
 (65.7, 88.68),
 (140.78, 200.67),
 (83.26, 114.3),
 (122.87, 173.46),
 (118.64, 167.06),
 (98.83, 137.36),
 (174.25, 252.22),
 (112.82, 158.29),
 (126.06, 178.28),
 (43.08, 56.41),
 (86.25, 118.71),
 (59.23, 79.35),
 (106.84, 149.31),
 (60.29, 80.88),
 (113.35, 159.09),
 (93.76, 129.81),
 (87.03, 119.85),
 (121.13, 170.82),
 (112.64, 158.02),
 (94.93, 131.55),
 (112.41, 157.68),
 (77.08, 105.23),
 (85.91, 118.2),
 (103.1, 143.72),
 (123.25, 174.03),
 (105.48, 147.29),
 (111.65, 156.54),
 (86.82, 119.55),
 (91.07, 125.84),
 (77.47, 105.8),
 (116.84, 164.34),
 (57.98, 77.56),
 (52.39, 69.57),
 (53.2, 70.74),
 (25.6, 32.3),
 (83.71, 114.96),
 (82.05, 112.52),
 (172.0, 248.74),
 (11

**Save Model and Params**

In [168]:
# Save best model and cv
save_load(best_model_cat_reg, title="best_model_cat_reg", function="save")
save_load(grid_cat_reg, title="best_cv_cat_reg", function="save")

## NN model 1: Neural Networks

In [169]:
# Build the model
#model_nn_seq = models.Sequential()

# Input Layer
#model_nn_seq.add(
#    layers.Dense(128,
#                 input_shape=(X_train_prep.shape[1], ),
#                 kernel_regularizer=regularizers.l1(0.005),
#                 activation='relu'))

# Hidden Layers
#model_nn_seq.add(
#    layers.Dense(256,
#                 kernel_regularizer=regularizers.l1(0.005),
#                 activation='relu'))
#model_nn_seq.add(
#    layers.Dense(256,
#                 kernel_regularizer=regularizers.l1(0.005),
#                 activation='relu'))
#model_nn_seq.add(
#    layers.Dense(512,
#                 kernel_regularizer=regularizers.l1(0.005),
#                 activation='relu'))

# Output Layer
#model_nn_seq.add(layers.Dense(1, activation='linear'))

# Compile the model
#model_nn_seq.compile(loss='mean_absolute_percentage_error',
#                     optimizer='adam',
#                     metrics=['mean_absolute_percentage_error'])

# Model summary
#print(model_nn_seq.summary())

# Visualize the neural network
#SVG(model_to_dot(model_nn_seq, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

In [170]:
# Create pipeline with preprocessing
#pipeline_nn_reg = Pipeline([('preprocessor', preprocessor),
#                             ('nn_reg',
#                              KerasRegressor(build_fn=model_nn_seq, epochs=20, batch_size=250))])

In [171]:
# Define hyperparameter grid
#param_grid_nn_reg = {
#    'xgb_reg__bootstrap': [True, False],
#    'nn_reg__epochs': [10, 20]
#}

In [172]:
# Create and fit GridSearchCV, save "best_model"
#grid_nn_reg = GridSearchCV(pipeline_nn_reg,
#                            param_grid_nn_reg,
#                            cv=5,
#                            scoring=scoring,
#                            return_train_score=True,
#                            verbose=4,
#                            n_jobs=-1)

#grid_nn_reg = grid_nn_reg.fit(X_train, y_train)

In [173]:
# Train the model
#model_nn_seq_start = time.time()

#best_model_nn_reg = model_nn_seq.fit(X_train_prep,
#                                        y_train,
#                                        epochs=20,
#                                        batch_size=256,
#                                        validation_split=0.2)

#model_nn_seq_end = time.time()

#print(f"Time taken to run: {round((model_nn_seq_end - model_nn_seq_start)/60,1)} minutes")

**Training Set**

In [174]:
# Predict target with "best model"
#y_train_pred_nn_reg = model_nn_seq.predict(X_train_prep).ravel()

In [175]:
# Final evaluation of "best model"
#model_eval(y_train, y_train_pred_nn_reg, model="reg")

**Testing Set**

In [176]:
# Predict target with "best model"
#y_test_pred_nn_reg = model_nn_seq.predict(X_test_prep).ravel()

In [177]:
# Final evaluation of "best model"
#model_eval(y_test, y_test_pred_nn_reg, model="reg")

In [178]:
# Display confidence interval (scipy stats)
#confidence = 0.95
#squared_errors = (y_test_pred_nn_reg - y_test)**2
#np.sqrt(
#    stats.t.interval(confidence,
#                     len(squared_errors) - 1,
#                     loc=squared_errors.mean(),
#                     scale=stats.sem(squared_errors)))

**Median Price Intervals**

In [179]:
# Save MAPE_median as variable
#MAPE_median_nn_reg = (median_absolute_percentage_error(y_test, y_test_pred_nn_reg))/100

In [180]:
# Calculate price interval for MAPE median
#y_pred_interval_nn_reg = tuple([(round(math.exp(el-el*MAPE_median_nn_reg),2),round(math.exp(el+el*MAPE_median_nn_reg),2)) for el in y_test_pred_nn_reg])
#y_pred_interval_nn_reg

In [181]:
# Evaluate the model
#model_nn_seq.model_evaluation(model_nn_seq, skip_epochs=2, X_train=X_train, X_test=X_test)

#score_nn_seq = model_nn_seq.evaluate(X_train_prep, y_train,verbose=1)
#print(score_nn_seq)

**Save Model and Params**

In [182]:
# Save best model and cv
#save_load(best_model_nn_reg, title="best_model_nn_reg", function="save")

# Findings and implications

**Findings**

- Data on Airbnb include a **substantial amount of inactive listings** which can be easily removed by checking availability in the next 365 days
- While it is not a trivial task to predict based only on features potentially available before listing creation, the **predictive model yielded solid results** and can definitely serve as a first indicator when considering a new listing
- Of the eventually used features, **room type** appeared to be the most important one, followed by **guests included**, **bedrooms** and **host listings** (i.e. whether the host already has prior listings)

**Implications**

- Even without actual booking data, it is possible to provide a **solid and relevant pricing indication** to new hosts
- When aiming for a high price, hosts should particularly **think about the room type** they offer (i.e. potentially consider renting out the whole flat or a private room rather than a shared room)
- Furthermore, the **location** (zipcode) plays a big role, but for a more precise indication of the surroundings it is suggested to have a look at nearby listings via the map in the web application

# Model selection for web application

**data_engineered**

In [111]:
# Load or assign data_engineered
#APP_data_engineered = save_load(title="data_engineered", function="load", dataset_loc=dataset_loc, dataset_date=dataset_date, model_run=model_run)
APP_data_engineered = data_engineered.copy()

In [112]:
# Save best model and cv
save_load(APP_data_engineered, title="APP_data_engineered", file_format="app", function="save")

**Best model**

In [113]:
# Load or assign best model
#APP_best_model = save_load(title="best_model_xgb_reg", function="load", dataset_loc=dataset_loc, dataset_date=dataset_date, model_run=model_run)
APP_best_model = best_model_xgb_reg

In [114]:
# Save best model and cv
save_load(APP_best_model, title="APP_best_model", file_format="app", function="save")

**Preprocessor**

In [115]:
# Load or assign preprocessor
#APP_preprocessor = save_load(title="preprocessor", function="load", dataset_loc=dataset_loc, dataset_date=dataset_date, model_run=model_run)
APP_preprocessor = X_train_prep_preprocessor

In [116]:
# Save preprocessor
save_load(APP_preprocessor, title="APP_preprocessor", file_format="app", function="save")

**X_test**

In [117]:
# Load or assign X_test
#APP_X_test = save_load(title="X_test", function="load", dataset_loc=dataset_loc, dataset_date=dataset_date, model_run=model_run)
APP_X_test = X_test

In [118]:
# Save X_test
save_load(APP_X_test, title="APP_X_test", file_format="app", function="save")

In [119]:
# Load or assign MAPE_median
#APP_MAPE_median = save_load(title="MAPE_median_xgb_reg", function="load", dataset_loc=dataset_loc, dataset_date=dataset_date, model_run=model_run)
APP_MAPE_median = MAPE_median_xgb_reg

**MAPE_median**

In [120]:
# Save MAPE_median
save_load(APP_MAPE_median, title="APP_MAPE_median", file_format="app", function="save")

**Zipcode**

In [121]:
# Load or assign zipcode
#APP_zipcode = save_load(title="MAPE_median_xgb_reg", function="load", dataset_loc=dataset_loc, dataset_date=dataset_date, model_run=model_run)
APP_zipcode = list(set(data.zipcode))
APP_zipcode.sort()

In [122]:
# Save zipcode
save_load(APP_zipcode, title="APP_zipcode", file_format="app", function="save")

# Future work

**Predictive modeling**
- Apply further models and adapt current ones (e.g. NN)
- Examine other prediction targets (e.g. occupancy rate)

**Feature engineering**
- Explore NLP for text fields (descriptions, reviews, ...)
- Scrape listing photos and analyze quality
- Enhance current feature set

**Lean structure**
- Remove remaining redundancies wherever possible (e.g. pack repeated steps into functions, apply more pipelines, ...)

**Cloud**
- Move both model creation and app into the cloud (GCP)

**Automatization and replicability**
- Build a workflow to automatically retrain model monthly with new datasets
- Use automated outlier detection
- Use automated feature engineering

**Replicability**
- Reduce redundancies wherever possible (e.g. pack repeated steps into functions, apply more pipelines, ...)
- Apply analysis to other cities and compare results